# Setup

In [50]:
# Set the year for the analysis
analysis_year = 2023
analysis_year_cra = 2022

In [2]:
# Load modules
import pandas as pd
import os
import numpy as np
# from utils import affiliate_institutions, institution_name_transform, bin                                                                                                                  
from utils import *

# Set pandas dataframe options
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [3]:
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory `{directory_path}` created successfully.")
    else:
        print(f"Directory `{directory_path}` already exists.")

In [4]:
# Define helper functions
def format_census_tract(tract_number, remove_decimal=False):
    if not tract_number or tract_number == '0000NAN':
        return tract_number
    
    # Convert to float
    tract_number = float(tract_number)
    
    # Format
    res = '%07.2F'% tract_number
    
    # Remove decimal
    if remove_decimal:
        return res.replace('.', '')
    else:
        return res



def sum_eth(row):
    return row['Asian'] + row['Black or African American'] +  row['Hispanic or Latino'] +  row['Other'] + row['White not Hispanic or Latino']


In [5]:
# Create folder for Excel exports
excel_exports_parent_folder = "excel-exports"

create_directory(excel_exports_parent_folder)

Directory `excel-exports` already exists.


In [6]:
# Create a subfolder for the analysis year in the Excel exports folder
excel_exports_folder = os.path.join(excel_exports_parent_folder, str(analysis_year))
create_directory(excel_exports_folder)

Directory `excel-exports/2023` already exists.


In [7]:
data_exports_parent_folder = "data-exports"

data_exports_folder = os.path.join(data_exports_parent_folder, str(analysis_year))
os.listdir(data_exports_folder)


['hmda_dict_lar_df.pkl',
 'fdic_locations_df.pkl',
 'fdic_institutions_df.pkl',
 'ffiec_data.pkl',
 'sba_data.pkl',
 'hmda_dict_panel_df.pkl']

# **Unpickle dfs from pickle files**

## Census

In [8]:
try:
    census_df = pd.read_pickle(os.path.abspath(os.path.join(data_exports_folder,'census.pkl')))
    census_df.head(5)
except Exception as e:
    print(e)

[Errno 2] No such file or directory: '/Users/korelasi/GitHub/southern_dallas_progress/banking_dashboard/data-exports/2023/census.pkl'


## FFIEC data

In [9]:
# Raw dataset
ffiec_data = pd.read_pickle(os.path.abspath(os.path.join(data_exports_folder,'ffiec_data.pkl')))

# FFIEC data preprocessing
ffiec_data['Tract Income Indicator'] = \
ffiec_data['Income indicator, which identifies low, moderate, middle, and upper income areas']\
.replace({0.:"Unknown Income", 1.:"Low or Moderate Income", 2.:"Low or Moderate Income",\
          3.:"Middle Income", 4.:"Upper Income"}) 

ffiec_data.head(5)

HMDA/CRA collection year MSA/MD Code  State         County Census tract  \
72753                     2023       19124  TEXAS  Collin County      0301.01   
72754                     2023       19124  TEXAS  Collin County      0301.02   
72755                     2023       19124  TEXAS  Collin County      0302.01   
72756                     2023       19124  TEXAS  Collin County      0302.02   
72757                     2023       19124  TEXAS  Collin County      0302.04   

      Principal city flag Small county flag  \
72753  not principal city      tract record   
72754  not principal city      tract record   
72755  not principal city      tract record   
72756  not principal city      tract record   
72757  not principal city      tract record   

                        Split tract flag  \
72753  tract number occurs within one MA   
72754  tract number occurs within one MA   
72755  tract number occurs within one MA   
72756  tract number occurs within one MA   
72757  tract number occurs within one MA   

                                   Demographic data flag Urban/rural flag  \
72753  total persons/population and median family inc...            rural   
72754  total persons/population and median family inc...            rural   
72755  total persons/population and median family inc...            mixed   
72756  total persons/population and median family inc...            rural   
72757  total persons/population and median family inc...            mixed   

       MSA/MD median family income  MSA/MD median household income  \
72753                      88315.0                         74251.0   
72754                      88315.0                         74251.0   
72755                      88315.0                         74251.0   
72756                      88315.0                         74251.0   
72757                      88315.0                         74251.0   

       Tract median family income as a percentage of the MSA/MD median family income. 2 decimal places, truncated.  \
72753                                              89.77                                                             
72754                                              76.70                                                             
72755                                             100.49                                                             
72756                                             163.98                                                             
72757                                             112.05                                                             

       FFIEC Estimated MSA/MD median family income  \
72753                                     105600.0   
72754                                     105600.0   
72755                                     105600.0   
72756                                     105600.0   
72757                                     105600.0   

       Income indicator, which identifies low, moderate, middle, and upper income areas  \
72753                                                3.0                                  
72754                                                2.0                                  
72755                                                3.0                                  
72756                                                4.0                                  
72757                                                3.0                                  

      CRA poverty criteria CRA unemployment criteria CRA distressed criteria  \
72753                   No                        No                      No   
72754                   No                        No                      No   
72755                   No                        No                      No   
72756                   No                        No                      No   
72757                   No                        No                      No   

      CRA remote rural (l

In [10]:
# Create a lookup table for FFIEC tract-level income indicators
ffiec_tract_income_lookup = ffiec_data[['State', 'County', 'Census tract', 'Tract Income Indicator']].drop_duplicates().reset_index()

# Insert census tract ID with decimal removed
ffiec_tract_income_lookup['Census_Tract_No_Decimal'] = ffiec_tract_income_lookup['Census tract'].apply(lambda x: x.replace(".", ""))

ffiec_tract_income_lookup.head(10)

,index,State,County,Census tract,Tract Income Indicator,Census_Tract_No_Decimal
0,72753,TEXAS,Collin County,0301.01,Middle Income,030101
1,72754,TEXAS,Collin County,0301.02,Low or Moderate Income,030102
2,72755,TEXAS,Collin County,0302.01,Middle Income,030201
3,72756,TEXAS,Collin County,0302.02,Upper Income,030202
4,72757,TEXAS,Collin County,0302.04,Middle Income,030204
5,72758,TEXAS,Collin County,0302.05,Middle Income,030205
6,72759,TEXAS,Collin County,0302.06,Upper Income,030206
7,72760,TEXAS,Collin County,0302.07,Middle Income,030207
8,72761,TEXAS,Collin County,0303.01,Upper Income,030301
9,72762,TEXAS,Collin County,0303.02,Upper Income,030302


In [11]:
# Export FFIEC tract-level income indicators to Excel
out_file_path = os.path.join(excel_exports_folder, "ffiec_tract_income_lookup.xlsx")
export_to_excel(ffiec_tract_income_lookup, out_file_path, sheet_name="ffiec_tract_income_lookup", startcol=0, startrow=0)

Exporting dataframe to excel-exports/2023/ffiec_tract_income_lookup.xlsx


## HMDA - Loan Application Record (LAR)

In [12]:
# Raw data
hmda_dict_lar_df = pd.read_pickle(os.path.abspath(os.path.join(data_exports_folder,'hmda_dict_lar_df.pkl')))

hmda_dict_lar_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,census_tract_full
57,2023,549300JOT0D4J0SZIK67,19124,TEXAS,Dallas County,0190.24,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Freddie Mac,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,195000,NaN,5.875,NaN,Not a high-cost mortgage,6600.9,NaN,3880.0,665.0,NaN,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,455000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,8888,8888,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,4164,58.29,105600,110.56,1207,1498,0,NaN,48113019024.0
122,2023,549300JOT0D4J0SZIK67,23104,TEXAS,Tarrant County,1113.19,NaN,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Ginnie Mae,Preapproval not requested,Veterans Affairs guaranteed (VA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,745000,NaN,4.75,NaN,Not a high-cost mortgage,12082.02,NaN,5975.0,4980.0,265.45,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,745000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN

In [13]:


# List of categorical variables
category_lst = ['derived_loan_product_type', 'loan_type','loan_purpose','lien_status',\
                'open_end_line_of_credit','business_or_commercial_purpose',\
                'debt_to_income_ratio','applicant_credit_score_type','applicant_sex',\
                'applicant_age']

# Combine low frequency categories
bin_category(hmda_dict_lar_df, category_lst, 200)

# binning certain values to further preprocess hmda data 
dti_replace_map = {dti_val:'36%-<50%' for dti_val in hmda_dict_lar_df['debt_to_income_ratio']  if str(dti_val) in str(set(range(36,50)))}
hmda_dict_lar_df['debt_to_income_ratio'] = hmda_dict_lar_df['debt_to_income_ratio'].replace(dti_replace_map).replace({np.nan:'Unknown'})
hmda_dict_lar_df['applicant_age'] = hmda_dict_lar_df['applicant_age'].replace({'8888':'Not applicable'})

hmda_dict_lar_df.head(5)

       derived_loan_product_type   count
0        Conventional:First Lien  119388
1  Conventional:Subordinate Lien   29577
2                 FHA:First Lien   26796
3                  VA:First Lien    8955
4       FSA/RHS:Subordinate Lien      58
                                           loan_type   count
0  Conventional (not insured or guaranteed by FHA...  148965
1       Federal Housing Administration insured (FHA)   26816
2                   Veterans Affairs guaranteed (VA)    8957
3  USDA Rural Housing Service or Farm Service Age...     110
           loan_purpose   count
0         Home purchase  121957
1  Cash-out refinancing   22961
2         Other purpose   15089
3      Home improvement   13437
4           Refinancing   10486
                     lien_status   count
0        Secured by a first lien  155191
1  Secured by a subordinate lien   29657
          open_end_line_of_credit   count
0  Not an open-end line of credit  161444
1         Open-end line of credit   21815
2       

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,census_tract_full
57,2023,549300JOT0D4J0SZIK67,19124,TEXAS,Dallas County,0190.24,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Freddie Mac,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,195000,NaN,5.875,NaN,Not a high-cost mortgage,6600.9,NaN,3880.0,665.0,NaN,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,455000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,8888,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,4164,58.29,105600,110.56,1207,1498,0,NaN,48113019024.0
122,2023,549300JOT0D4J0SZIK67,23104,TEXAS,Tarrant County,1113.19,NaN,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Ginnie Mae,Preapproval not requested,Veterans Affairs guaranteed (VA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,745000,NaN,4.75,NaN,Not a high-cost mortgage,12082.02,NaN,5975.0,4980.0,265.45,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,745000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not appl

### HMDA Panel Data

In [15]:
hmda_dict_panel_df = pd.read_pickle(os.path.abspath(os.path.join(data_exports_folder,'hmda_dict_panel_df.pkl')))
hmda_dict_panel_df.head(5)

,activity_year,lei,tax_id,agency_code,id_2017,respondent_rssd,respondent_name,respondent_state,respondent_city,assets,other_lender_code,parent_rssd,parent_name,topholder_rssd,topholder_name
0,2023,5493003GQDUH26DNNH17,53-0116705,Consumer Financial Protection Bureau,617677,617677.0,NAVY FEDERAL CREDIT UNION,VA,Vienna,1.566451e+08,Depository Institution,NaN,NaN,NaN,NaN
1,2023,549300KHXD7JSQUZIJ22,75-2269700,Department of Housing and Urban Development,4437770,4437770.0,"HIGHLANDS RESIDENTIAL MORTGAGE, LTD.",TX,Dallas,NaN,Independent mortgage banking subsidiary,NaN,NaN,NaN,NaN
2,2023,5493001SXWZ4OFP8Z903,74-2853239,Department of Housing and Urban Development,3332551,3332551.0,"DHI MORTGAGE COMPANY, LTD.",TX,DALLAS,NaN,MBS of bank holding company,NaN,NaN,NaN,NaN
3,2023,549300KILAW2M87QR043,35-1049024,National Credit Union Administration,33389,33389.0,INDIANA STATE UNIVERSITY FEDERAL CREDIT UNION,IN,Terre Haute,1.317130e+05,Depository Institution,NaN,NaN,NaN,NaN
4,2023,7H6GLXDRUGQFU57RNE97,13-4994650,Consumer Financial Protection Bureau,852218,852218.0,"JPMorgan Chase Bank, National Association",OH,Columbus,3.201942e+09,Depository Institution,1039502.0,JPMORGAN CHASE & CO. ...,1039502.0,JPMORGAN CHASE & CO. ...


In [16]:
hmda_dict_panel_df['agency_code'].value_counts()

National Credit Union Administration           1575
Federal Deposit Insurance Corporation          1568
Department of Housing and Urban Development     907
Office of the Comptroller of the Currency       528
Federal Reserve System                          396
Consumer Financial Protection Bureau            139
Name: agency_code, dtype: int64

# CRA Data
- Transmittal Sheet
- Disclosure D1-1 Data

Joined on respondent ID and agency code

In [51]:
# CRA data folder, with analysis_year_cra
cra_data_folder = os.path.join(data_exports_parent_folder, str(analysis_year_cra))
cra_data_folder

'data-exports/2022'

In [53]:
# Transmittal sheet
cra_2022_transmittal_sheet = pd.read_pickle(os.path.abspath(os.path.join(cra_data_folder,'CRA2022_Transmittal.pkl')))
cra_2022_transmittal_sheet.head(5)

,Respondent ID,Agency Code,Activity Year,Institution name,Institution address,Institution city,Institution state,Institution zip code,Tax ID,ID_RSSD,Institution assets
0,0000000001,OCC,2022,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
1,0000000008,OCC,2022,"JPMORGAN CHASE BANK, NA",1111 POLARIS PARKWAY,COLUMBUS,OH,43240,13-4994650,852218,3306982000
2,0000000024,OCC,2022,US BANK NA,777 E WISCONSIN AVE,MILWAUKEE,WI,53202,31-0841368,504713,564154604
3,0000000047,OCC,2022,"FIRST FINANCIAL BANK, NA","1401 S THIRD ST, PO BOX 2122",TERRE HAUTE,IN,47802,35-0704860,693345,5045573
4,0000000209,OCC,2022,FIRST NATIONAL BANK OF OMAHA,1620 DODGE STREET,OMAHA,NE,68197,47-0259043,527954,26812859


In [54]:
# CRA Discl D11
cra2022_Discl_D11_df_raw = pd.read_pickle(os.path.abspath(os.path.join(cra_data_folder,'cra2022_Discl_D11.pkl')))
cra2022_Discl_D11_df_raw.head(5)

,Table ID,Respondent ID,Agency Code,Activity Year,Loan Type,Action Taken Type,State,County,MSA/MD,Assessment Area Number,Partial County Indicator,Split County Indicator,Population Classification,Income Group Total,Report Level,"Number of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Number of Small Business Loans Originated with Loan Amount at Origination > 100,000 and < or = to $250,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $100,000 and < or = to $250,000","Number of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000",Number of Loans Originated to Small Businesses with Gross Annual Revenues < $1 million,Total Loan Amount of Loans Originated to Small Businesses with Gross Annual Revenues < or = to $1 million,Number of Small Business Loans Originated Reported as Affiliate Loans,Total Loan Amount of Small Business Loans Originated Reported as Affiliate Loans
17073,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",< 10% of Median Family Income(MFI),not a total,0,0,0,0,0,0,0,0,0,0
17074,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",10% to 20% of MFI,not a total,0,0,0,0,0,0,0,0,0,0
17075,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",20% to 30% of MFI,not a total,0,0,0,0,0,0,0,0,0,0
17076,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",30% to 40% of MFI,not a total,0,0,0,0,0,0,0,0,0,0
17077,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,not a total,1,91000,1,102000,0,0,0,0,2,193000


In [55]:
# Join transmittal sheet to 2022 cra d11 file 
cra2022_Discl_D11_df = pd.merge(cra2022_Discl_D11_df_raw,
         cra_2022_transmittal_sheet,
         left_on = ['Respondent ID','Agency Code', 'Activity Year'],
         right_on = ['Respondent ID','Agency Code', 'Activity Year'],
         how = 'left')


cra2022_Discl_D11_df.head(5)

,Table ID,Respondent ID,Agency Code,Activity Year,Loan Type,Action Taken Type,State,County,MSA/MD,Assessment Area Number,Partial County Indicator,Split County Indicator,Population Classification,Income Group Total,Report Level,"Number of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Number of Small Business Loans Originated with Loan Amount at Origination > 100,000 and < or = to $250,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $100,000 and < or = to $250,000","Number of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000",Number of Loans Originated to Small Businesses with Gross Annual Revenues < $1 million,Total Loan Amount of Loans Originated to Small Businesses with Gross Annual Revenues < or = to $1 million,Number of Small Business Loans Originated Reported as Affiliate Loans,Total Loan Amount of Small Business Loans Originated Reported as Affiliate Loans,Institution name,Institution address,Institution city,Institution state,Institution zip code,Tax ID,ID_RSSD,Institution assets
0,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",< 10% of Median Family Income(MFI),not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
1,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",10% to 20% of MFI,not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
2,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",20% to 30% of MFI,not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
3,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",30% to 40% of MFI,not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
4,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,not a total,1,91000,1,102000,0,0,0,0,2,193000,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000


In [56]:
# subset of cra data for dashboard
cra_d11_cols = [i for i in cra2022_Discl_D11_df_raw.columns.to_list() if 'Number of ' in i or 'Total Loan ' in i]

cra_2022_D11_subset = cra2022_Discl_D11_df[[
    'Respondent ID',
    'Agency Code',
    'Institution name',
    'Institution address',
    'Institution city',
    'Institution state',
    'Institution zip code',
    'Institution assets',
    'County',
    'Population Classification',
    'Income Group Total']\
+ cra_d11_cols].reset_index()

# reset index to list counting from 0 to len of range of original index col
cra_2022_D11_subset['index'] = list(range(0,len(cra_2022_D11_subset['index'])))

# set index on new index column
cra_2022_D11_subset = cra_2022_D11_subset.set_index('index')

# get indices of full d11 dataframe as set
all_d11_indices = set(cra_2022_D11_subset.index.to_list())

# get indices of subset d11 df containing rows that have totals as entries
d11_totals_indices = set(cra_2022_D11_subset[cra_2022_D11_subset['Population Classification'] == 'total'].index.to_list())

# find indices from full d11 df that do not have totals entires
d11_indices_no_totals = list(all_d11_indices - d11_totals_indices)
cra_2022_D11_subset = cra_2022_D11_subset.iloc[d11_indices_no_totals,:]

cra_2022_D11_subset.head(5)


,Respondent ID,Agency Code,Institution name,Institution address,Institution city,Institution state,Institution zip code,Institution assets,County,Population Classification,Income Group Total,"Number of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Number of Small Business Loans Originated with Loan Amount at Origination > 100,000 and < or = to $250,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $100,000 and < or = to $250,000","Number of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000",Number of Loans Originated to Small Businesses with Gross Annual Revenues < $1 million,Total Loan Amount of Loans Originated to Small Businesses with Gross Annual Revenues < or = to $1 million,Number of Small Business Loans Originated Reported as Affiliate Loans,Total Loan Amount of Small Business Loans Originated Reported as Affiliate Loans
index,,,,,,,,,,,,,,,,,,,,,
0,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",< 10% of Median Family Income(MFI),0,0,0,0,0,0,0,0,0,0
1,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",10% to 20% of MFI,0,0,0,0,0,0,0,0,0,0
2,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",20% to 30% of MFI,0,0,0,0,0,0,0,0,0,0
3,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",30% to 40% of MFI,0,0,0,0,0,0,0,0,0,0
4,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",40% to 50% of MFI,1,91000,1,102000,0,0,0,0,2,193000


## CRA Disclosure D-6

In [57]:
# Raw data
cra2022_Discl_D6_df_raw = pd.read_pickle(os.path.abspath(os.path.join(cra_data_folder,'cra2022_Discl_D6.pkl')))

# Merge with transmittal sheet
cra2022_Discl_D6_df = pd.merge(cra2022_Discl_D6_df_raw,
         cra_2022_transmittal_sheet,
         left_on = ['Respondent ID', 'Agency Code', 'Activity Year'],
         right_on = ['Respondent ID', 'Agency Code', 'Activity Year'],
         how = 'left')

cra2022_Discl_D6_df.head(5)

,Table ID,Respondent ID,Agency Code,Activity Year,State,County,MSA/MD,Census Tract,Assessment Area Number,Partial County Indicator,Split County Indicator,Population Classification,Income Group,Loan Indicator,Filler,Institution name,Institution address,Institution city,Institution state,Institution zip code,Tax ID,ID_RSSD,Institution assets
0,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0317.20,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
1,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0317.23,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,No,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
2,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0304.09,427.0,No,No,"counties with >500,000 in population",50% to 60% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
3,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0320.12,427.0,No,No,"counties with >500,000 in population",50% to 60% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
4,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0305.40,427.0,No,No,"counties with >500,000 in population",60% to 70% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000


In [58]:
# Income group lookup
sel_cols = ['County', 'Census Tract', 'Income Group']
cra_income_group_lookup = cra2022_Discl_D6_df[sel_cols].drop_duplicates()
cra_income_group_lookup = cra_income_group_lookup.sort_values(sel_cols).reset_index(drop=True)
# Rename cra_income_group_lookup 'Income Group' column to 'CRA Tract Income Group'
cra_income_group_lookup.rename(columns={'Income Group': 'CRA Tract Income Group'}, inplace=True)
cra_income_group_lookup.head(5)

,County,Census Tract,CRA Tract Income Group
0,Collin County,0301.01,80% to 90% of MFI
1,Collin County,0301.02,70% to 80% of MFI
2,Collin County,0302.01,100% to 110% of MFI
3,Collin County,0302.02,> 120% of MFI
4,Collin County,0302.04,110% to 120% of MFI


## FDIC Institutions & Locations

In [17]:
# FDIC institutions
fdic_institutions_df = pd.read_pickle(os.path.abspath(os.path.join(data_exports_folder,'fdic_institutions_df.pkl')))
fdic_institutions_df['FDIC Certificate #'] = fdic_institutions_df['FDIC Certificate #'].apply(float)
fdic_institutions_df.head(5)

,FDIC Certificate #,index,State Name,OTS Docket Number,Institution Status,Street Address,Total assets,Institution Class,Change Code 1,Change Code 2,Change Code 3,Change Code 4,Change Code 5,Change Code 6,Change Code 7,Change Code 8,Change Code 9,Change Code 10,Change Code 11,Change Code 12,Change Code 13,Change Code 14,Change Code 15,OCC Charter Number,Chartering Agency,Conservatorship,City,Numeric Code,Consolidated Metropolitan Statistical Division Number,Consolidated Metropolitan Statistical Area,County,Last update,Denovo Institution,Total deposits,Last Structure Change Effective Date,End date,Equity capital,Established Date,FDIC Geographic Region,FDIC Supervisory Region,Federal Reserve District,Federal Reserve ID Number,Federal Reserve ID Number 2,Federal charter flag,FDIC Field Office,Insured offices of foreign banks,Inactive,Primary Insurance Agency,Secondary Insurance Fund,Date of Deposit Insurance,Credit Card Institutions,Bank Insurance Fund,Insured commercial banks,Deposit Insurance Fund member,FDIC Insured,SAIF Insured,Insured Savings Institution,Metropolitan Statistical Area Number,Metropolitan Statistical Area (MSA),Institution name,New certificate number,Oakar Institutions,OTS District,Office of Thrift Supervision Region,Last Structure Change Process Date,Quarterly Banking Profile Commercial Bank Region,Primary Regulator,Report Date,Report Date,State Charter,Return on assets (ROA),Quarterly return on assets,Return on Equity (ROE),Quarterly return on equity,Run Date,Sasser Institutions,Law Sasser Flag,State Alpha code,State and county number,State Number,Zip Code,Supervisory Region Number,Office of the Comptroller,FDIC's unique number,Ultimate Cert,CFPB Effective Date,CFPB End Date,CFPB Flag,Secondary Regulator,Web Site URL 01,Web Site URL 02,Web Site URL 03,Web Site URL 04,Web Site URL 05,Web Site URL 06,Web Site URL 07,Web Site URL 08,Web Site URL 09,Web Site URL 10,Trade Name 01,Trade Name 02,Trade Name 03,Trade Name 04,Trade Name 05,Trade Name 06,Primary Internet Web Address,Office,Directly owned by another bank (CERT),Directly owned by another bank (CERT),City of High Holder,Deposits held in domestic offices,FFIEC Call Report 31 Filer,Bank Holding Company Type,Agricultural lending institution indicator,Ownership Type,Bank Holding Company (Regulatory Top Holder),Net income,Net income - quarterly,Number of Domestic Offices,Number of Foreign Offices,Number of US Offices,RSSDID - High Regulatory Holder,Regulatory holding company state location,Subchapter S Corporations,Pretax return on assets,Quarterly Pretax return on assets,Trust Powers,Asset Concentration Hierarchy,Specialization Group,Institutions with reportable fiduciary related service,Combined Statistical Area Name,Combined Statistical Area Number (Branch),CSA Area Flag,Core Based Statistical Area Name,Core Based Statistical Areas,Metropolitan Division Name,Metropolitan Division Number,Metropolitan Division Flag,Micropolitan Division Flag,Metropolitan Divisions Name,Metropolitan Divisions Number,Metropolitan Divisions Flag,Community Bank
0,19712.0,363,Texas,0,Institution closed or not insured by FDIC,1205 South Main Street,37128.0,"Commercial bank, state charter, Fed non-member...",223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,STATE,NaN,Grapevine,NaN,31,"Dallas-Fort Worth, TX",Tarrant,08/19/1993,0.0,34966.0,07/31/1993,07/31/1993,1870.0,1967-02-02,13,Dallas,Dallas,Dallas,457154,0,Dallas,0,1,BIF,NaN,06/01/1967,0.0,1,1,No,1,0,0,2800,"Fort Worth-Arlington, TX",Texas Bank,15832,0,5.0,West,08/19/1993,5,Federal Deposit Insurance Corporation,06/30/1993,03/31/2024,yes,0.929936,1.58,16.33,32.04,08/23/2024,0,NaN,TX,48439,48,76051,Dallas,5,13508,6384,9999-12-31,9999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,34966.0,no,NaN,0.0,0.0,NaN,170.0,144.0,2.0,0.0,0.0,NaN,NaN,0.0,0.929936,1.58,Trust Powers Not Known,4.0,Commercial Lending Specialization,NaN,"Dallas-Fort Worth, TX-OK",206.0,NaN,"Dallas-Fort 

In [18]:
# FDIC locations
fdic_locations_df = pd.read_pickle(os.path.abspath(os.path.join(data_exports_folder,'fdic_locations_df.pkl')))
fdic_locations_df.head(5)

,index,Branch Address,Institution Class,Core Based Statistical Area Name (Branch),Metropolitan Divisions Name (Branch),Metropolitan Divisions Flag (Branch),Metropolitan Divisions Number (Branch),Metropolitan Division Number (Branch),Metropolitan Division Flag (Branch),Metropolitan Division Name (Branch),Micropolitan Division Flag (Branch),Core Based Statistical Areas (Branch),Institution FDIC Certificate #,Branch City,Branch County,Combined Statistical Area Name (Branch),Combined Statistical Area Flag (Branch),Combined Statistical Area Number (Branch),Branch Established Date,FDIC UNINUM of the Owner Institution,Main Office,Institution Name,Office Name,Branch Number,Run Date,Service Type Code,Branch State Abbreviation,State and County Number,Branch State,Unique Identification Number for a Branch Office,Branch Zip Code,County_from_Zipcode,idx_branch_number,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,Branch identifier,Branch street address,Match result,Match type
0,33,1650 N Central Expy,"Commercial bank, state charter and Fed nonmemb...","Dallas-Fort Worth-Arlington, TX","Dallas-Plano-Irving, TX",Yes,19124.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,3076,Mckinney,Collin County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2009-10-26,1958,0,Independent Bank,McKinney Branch,24,08/23/2024,NaN,TX,48085,Texas,510842,75070,Collin County,(33) 24,TEXAS,0307.01,2010,Collin County,33.21325608364782,-96.63674285248845,{'latitute': '33.21325608364782'},(33) 24,"1650 N Central Expy, Mckinney, TX, 75070",Match,Exact
1,104,5800 Camp Bowie Blvd,"Commercial bank, state charter and Fed member,...","Dallas-Fort Worth-Arlington, TX","Fort Worth-Arlington-Grapevine, TX",Yes,23104.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,12614,Fort Worth,Tarrant County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2012-03-05,8010,0,Origin Bank,CAMP BOWIE BANKING CENTER,41,08/23/2024,NaN,TX,48439,Texas,529509,76107,Tarrant County,(104) 41,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109,301 Commerce St,"Commercial bank, state charter and Fed nonmemb...","Dallas-Fort Worth-Arlington, TX","Fort Worth-Arlington-Grapevine, TX",Yes,23104.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,20099,Fort Worth,Tarrant County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2012-05-29,13808,0,Texas Exchange Bank,DOWNTOWN FORT WORTH-SUNDANCE SQUARE,2,08/23/2024,NaN,TX,48439,Texas,531346,76102,Tarrant County,(109) 2,TEXAS,1233.02,2044,Tarrant County,32.75551876861715,-97.33067363451613,{'latitute': '32.75551876861715'},(109) 2,"301 Commerce St, Fort Worth, TX, 76102",Match,Exact
3,111,600 W Southlake Blvd,"Commercial bank, state charter and Fed nonmemb...","Dallas-Fort Worth-Arlington, TX","Fort Worth-Arlington-Grapevine, TX",Yes,23104.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,110,Southlake,Tarrant County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2012-07-02,89,0,Bank OZK,SOUTHLAKE BRANCH,145,08/23/2024,NaN,TX,48439,Texas,531684,76092,Tarrant County,(111) 145,TEXAS,1139.06,1011,Tarrant County,32.94129507399049,-97.16278977188585,{'latitute': '32.94129507399049'},(111) 145,"600 W Southlake Blvd, Southlake, TX, 76092",Match,Exact
4,116,2200 W 7th St,"Commercial bank, state charter and Fed member,...","Dallas-Fort Worth-Arlington, TX","Fort Worth-Arlington-Grapevine, TX",Yes,23104.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,3890,Fort Worth,Tarrant County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2012-03-12,2536,0,Simmons Bank,Fort Worth 7th Street Branch,251,08/23/2024,NaN,TX,48439,Texas,531958,76107,Tarrant County,(116) 251,TEXAS,1020.00,1058,Tarrant County,32.75108826152689,-97.3490231529089,{'latitute': '32.75108826152689'},(116) 251,"2200 W 7th St, Fort Worth, TX, 76107",Match,Exact


## SBA Data

In [32]:
# Raw data
sba_data = pd.read_pickle(os.path.abspath(os.path.join(data_exports_folder,'sba_data.pkl')))
sba_data.head(5)

,index,asofdate,program,Borrower name,Borrower street address,Borrower city,Borrower state,Borrower zip code,bankname,bankfdicnumber,bankncuanumber,bankstreet,bankcity,bankstate,bankzip,Total loan amount,sbaguaranteedapproval,approvaldate,approvalfiscalyear,firstdisbursementdate,deliverymethod,subprogram,initialinterestrate,fixedorvariableinterestind,terminmonths,naicscode,naicsdescription,franchisecode,franchisename,projectcounty,projectstate,sbadistrictoffice,congressionaldistrict,businesstype,businessage,loanstatus,paidinfulldate,chargeoffdate,grosschargeoffamount,revolverstatus,jobssupported,SOLDSECMRTIND,catalog_month,catalog_year,County_from_Zipcode,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,borrower identifier,borrower street address,Match result,Match type,county_final
4,480,6/30/2024,7A,(480) Spartan 6S Solutions LLC,9800 Hillwood Pkwy. Suite 14,Fort Worth,TEXAS,76177,"Colonial Savings, F.A.",31361,None,2600 West Frwy,FORT WORTH,TX,76102,500000.0,250000.0,2023-01-03,2023,1/3/2023,SBA Express,FA$TRK (Small Loan Express),8.50,V,84,484110.0,"General Freight Trucking, Local",NaN,NaN,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,12.0,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Revolver,141,NaN,7,2024,Tarrant County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant County
8,906,6/30/2024,7A,(906) Tameap LLC,9749 N Freeway,Fort Worth,TEXAS,76177,Dogwood State Bank,57095,None,5401 Six Forks Rd,RALEIGH,NC,27609,747300.0,560475.0,2023-01-04,2023,1/23/2023,Preferred Lender Program,Guaranty,10.25,V,126,722515.0,Snack and Nonalcoholic Beverage Bars,S2849,Just Love Coffee Cafe,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,12.0,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Term,22,Sold on the secondary market,7,2024,Tarrant County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant County
12,990,6/30/2024,7A,(990) Tameap LLC,9749 N Freeway,Fort Worth,TEXAS,76177,Dogwood State Bank,57095,None,5401 Six Forks Rd,RALEIGH,NC,27609,35000.0,17500.0,2023-01-04,2023,1/23/2023,SBA Express,FA$TRK (Small Loan Express),10.25,V,120,722515.0,Snack and Nonalcoholic Beverage Bars,S2849,Just Love Coffee Cafe,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,12.0,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Revolver,22,NaN,7,2024,Tarrant County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant County
19,1547,6/30/2024,7A,(1547) Elite Performance Chiropractic,6220 Virginia Parkway,MCKINNEY,TEXAS,75071,First United Bank and Trust Company,4239,None,1400 W Main St,DURANT,OK,74701,1624600.0,1218450.0,2023-01-05,2023,1/30/2023,Preferred Lender Program,Guaranty,9.25,V,300,621310.0,Offices of Chiropractors,NaN,NaN,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,3.0,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Term,15,NaN,7,2024,Collin County,TEXAS,0305.47,3023,Collin County,33.199333920070615,-96.70156262879846,{'latitute': '33.199333920070615'},(1547) Elite Performance Chiropractic,"6220 Virginia Parkway, MCKINNEY, TEXAS, 75071",Match,Exact,Collin County
29,2352,6/30/2024,7A,(2352) 3Kims 22 Inc,311 S ZANG BLVD,DALLAS,TEXAS,75208,Cadence Bank,11813,None,201 S Spring St,TUPELO,MS,38804,200000.0,150000.0,2023-01-06,2023,2/8/2023,Preferred Lender Program,Guaranty,10.25,V,120,722513.0,Limited-Service Restaurants,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,30.0,CORPORATION,NaN,Cancelled,NaN,NaN,0,Term,2,NaN,7,2024,Dallas County,TEXAS,0050.00,2002,Dallas County,32.74299266826324,-96.82470209042083,{'latitute': '32.74299266826324'},(2352) 3Kims 22 Inc,"311 S ZANG BLVD, DALLAS, TEXAS, 75208",Match,Exact,Dallas County


In [25]:
# Load SBA data from the previous analysis year
previous_analysis_year = analysis_year - 1
previous_analysis_year_folder = os.path.join(data_exports_parent_folder, str(previous_analysis_year))
previous_sba_data = pd.read_pickle(os.path.abspath(os.path.join(previous_analysis_year_folder, 'sba_data.pkl')))
previous_sba_data.head(5)

,index,AsOfDate,Program,Borrower name,Borrower street address_x,Borrower city,Borrower state,Borrower zip code,BankName,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,Total loan amount,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,Subprogram description,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SOLDSECMRTIND,County_from_Zipcode,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,Borrower identifier,Borrower street address_y,Match result,Match type,county_final
1,103084,20230930,7A,(103084) GHIJ LLC,4802 Carolina Circle,MCKINNEY,TEXAS,75071,Centennial Bank,11241.0,NaN,620 Chestnut St,CONWAY,AR,72032,100000.0,50000.0,2022-01-03,2022,NaN,SBA Express,FA$TRK (Small Loan Express),4.25,12,238170.0,Siding Contractors,S2684,Spray Net,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,3.0,CORPORATION,New Business or 2 years or less,Cancelled,NaN,NaN,0,Revolver,10,NaN,Collin County,TEXAS,0305.48,2011,Collin County,33.199346203703385,-96.67829742417973,{'latitute': '33.199346203703385'},(103084) GHIJ LLC,"4802 Carolina Circle, MCKINNEY, TEXAS, 75071",Match,Exact,Collin County
2,103088,20230930,7A,(103088) GHIJ LLC,4802 Carolina Circle,MCKINNEY,TEXAS,75071,Centennial Bank,11241.0,NaN,620 Chestnut St,CONWAY,AR,72032,201000.0,100500.0,2022-01-03,2022,NaN,SBA Express,FA$TRK (Small Loan Express),5.50,126,238170.0,Siding Contractors,S2684,Spray Net,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,3.0,CORPORATION,New Business or 2 years or less,Cancelled,NaN,NaN,0,Term,10,NaN,Collin County,TEXAS,0305.48,2011,Collin County,33.199346203703385,-96.67829742417973,{'latitute': '33.199346203703385'},(103088) GHIJ LLC,"4802 Carolina Circle, MCKINNEY, TEXAS, 75071",Match,Exact,Collin County
14,103266,20230930,7A,(103266) Forester Haynie PLLC,400 SAINT PAUL ST Suite 700,DALLAS,TEXAS,75201,Texas Security Bank,58739.0,NaN,1212 Turtle Creek Blvd,DALLAS,TX,75207,200000.0,100000.0,2022-01-04,2022,01/28/2022,SBA Express,FA$TRK (Small Loan Express),5.25,12,541110.0,Offices of Lawyers,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,30.0,CORPORATION,Existing or more than 2 years old,Paid In Full,05/31/2022,NaN,0,Revolver,3,NaN,Dallas County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dallas County
18,103339,20230930,7A,(103339) Aspire Technical Solutions LL,1475 HERITAGE PKWY SUITE 201,MANSFIELD,TEXAS,76063,Celtic Bank Corporation,57056.0,NaN,"268 S State St, Ste 300",SALT LAKE CITY,UT,84111,150000.0,127500.0,2022-01-05,2022,01/14/2022,Preferred Lender Program,Guaranty,6.00,120,541512.0,Computer Systems Design Services,NaN,NaN,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,6.0,CORPORATION,Existing or more than 2 years old,The status of loans that have been disbursed b...,NaN,NaN,0,Term,2,Sold on the secondary market,Tarrant County,TEXAS,1113.09,2003,Tarrant County,32.55325719311065,-97.12222046413416,{'latitute': '32.55325719311065'},(103339) Aspire Technical Solutions LL,"1475 HERITAGE PKWY SUITE 201, MANSFIELD, TEXAS...",Match,Exact,Tarrant County
19,103341,20230930,7A,(103341) Kmandy Investments L.L.C.,6509 San Judas Street,McKinney,TEXAS,75070,Celtic Bank Corporation,57056.0,NaN,"268 S State St, Ste 300",SALT LAKE CITY,UT,84111,586700.0,440025.0,2022-01-05,2022,01/11/2022,Preferred Lender Program,Guaranty,6.00,120,454210.0,Vending Machine Operators,S2056,WaterStation,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,3.0,CORPORATION,"Startup, Loan Funds will Open Business",The status of loans that have been disbursed b...,NaN,NaN,0,Term,1,Sold on the secondary market,Collin County,TEXAS,0305.40,2006,Collin County,33.15427189246418,-96.70470560393005,{'latitute': '33.15427189246418'},(103341) Kmandy Investments L.L.C.,"6509 San Judas Street, McKinney, TEXA

In [34]:
sba_data['businessage'].value_counts()


Series([], Name: businessage, dtype: int64)

In [28]:
# Find corresponding column names in the previous SBA data
column_map = {}
for col in sba_data.columns:
    for prev_col in previous_sba_data.columns:
        if col.lower() == prev_col.lower():
            column_map[col] = prev_col
            break 
        
column_map  

{'index': 'index',
 'asofdate': 'AsOfDate',
 'program': 'Program',
 'Borrower name': 'Borrower name',
 'Borrower city': 'Borrower city',
 'Borrower state': 'Borrower state',
 'Borrower zip code': 'Borrower zip code',
 'bankname': 'BankName',
 'bankfdicnumber': 'BankFDICNumber',
 'bankncuanumber': 'BankNCUANumber',
 'bankstreet': 'BankStreet',
 'bankcity': 'BankCity',
 'bankstate': 'BankState',
 'bankzip': 'BankZip',
 'Total loan amount': 'Total loan amount',
 'sbaguaranteedapproval': 'SBAGuaranteedApproval',
 'approvaldate': 'ApprovalDate',
 'approvalfiscalyear': 'ApprovalFiscalYear',
 'firstdisbursementdate': 'FirstDisbursementDate',
 'deliverymethod': 'DeliveryMethod',
 'initialinterestrate': 'InitialInterestRate',
 'terminmonths': 'TermInMonths',
 'naicscode': 'NaicsCode',
 'naicsdescription': 'NaicsDescription',
 'franchisecode': 'FranchiseCode',
 'franchisename': 'FranchiseName',
 'projectcounty': 'ProjectCounty',
 'projectstate': 'ProjectState',
 'sbadistrictoffice': 'SBADistrict

In [29]:
# Rename columns in sba_data
sba_data.rename(columns=column_map, inplace=True)

In [30]:
# subset for 'Paid In Full', 'Charged Off', and 'The status of loans that have been disbursed but have not been cancelled,
# paid in full, or charged off are exempt from disclosure under FOIA Exemption 4'

sba_data = sba_data[(sba_data['LoanStatus'] == 'Paid In Full') |\
(sba_data['LoanStatus'] == 'Charged Off') |\
(sba_data['LoanStatus'] == 'The status of loans that have been disbursed but have not been cancelled, paid in full, \
or charged off are exempt from disclosure under FOIA Exemption 4')]

print("recoding CongressionalDistrict as a string")
sba_data['CongressionalDistrict'] = sba_data['CongressionalDistrict'].apply(lambda dist: f"District {int(dist):02d}")

print("binning all unique entires within category column of sba data under .01")
sba_category_lst = ['DeliveryMethod','NaicsDescription','CongressionalDistrict','BusinessType','BusinessAge']
bin_category(sba_data, sba_category_lst, 0.01, True,'proportion')

# Rename bank name to institution name
sba_data = sba_data.rename(columns = {'BankName':'institution_name'})

sba_data.head(5)

             DeliveryMethod  proportion
0  Preferred Lender Program    0.736451
1               SBA Express    0.196599
2           Other 7(a) Loan    0.044633
3       Community Advantage    0.014878
4       International Trade    0.005313
                          NaicsDescription  proportion
0                 Full-Service Restaurants    0.071765
1              Limited-Service Restaurants    0.060000
2  Fitness and Recreational Sports Centers    0.041176
3     Snack and Nonalcoholic Beverage Bars    0.024706
4                   Residential Remodelers    0.021176
  CongressionalDistrict  proportion
0           District 03    0.295430
1           District 24    0.205101
2           District 12    0.131775
3           District 06    0.130712
4           District 30    0.102019
  BusinessType  proportion
0  CORPORATION    0.970244
1   INDIVIDUAL    0.014878
2  PARTNERSHIP    0.014878
Empty DataFrame
Columns: [BusinessAge, proportion]
Index: []


,index,AsOfDate,Program,Borrower name,Borrower street address,Borrower city,Borrower state,Borrower zip code,institution_name,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,Total loan amount,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,subprogram,InitialInterestRate,fixedorvariableinterestind,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SOLDSECMRTIND,catalog_month,catalog_year,County_from_Zipcode,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,Borrower identifier,borrower street address,Match result,Match type,county_final
4,480,6/30/2024,7A,(480) Spartan 6S Solutions LLC,9800 Hillwood Pkwy. Suite 14,Fort Worth,TEXAS,76177,"Colonial Savings, F.A.",31361,None,2600 West Frwy,FORT WORTH,TX,76102,500000.0,250000.0,2023-01-03,2023,1/3/2023,SBA Express,FA$TRK (Small Loan Express),8.50,V,84,484110.0,"General Freight Trucking, Local",NaN,NaN,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 12,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Revolver,141,NaN,7,2024,Tarrant County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant County
8,906,6/30/2024,7A,(906) Tameap LLC,9749 N Freeway,Fort Worth,TEXAS,76177,Dogwood State Bank,57095,None,5401 Six Forks Rd,RALEIGH,NC,27609,747300.0,560475.0,2023-01-04,2023,1/23/2023,Preferred Lender Program,Guaranty,10.25,V,126,722515.0,Snack and Nonalcoholic Beverage Bars,S2849,Just Love Coffee Cafe,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 12,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Term,22,Sold on the secondary market,7,2024,Tarrant County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant County
12,990,6/30/2024,7A,(990) Tameap LLC,9749 N Freeway,Fort Worth,TEXAS,76177,Dogwood State Bank,57095,None,5401 Six Forks Rd,RALEIGH,NC,27609,35000.0,17500.0,2023-01-04,2023,1/23/2023,SBA Express,FA$TRK (Small Loan Express),10.25,V,120,722515.0,Snack and Nonalcoholic Beverage Bars,S2849,Just Love Coffee Cafe,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 12,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Revolver,22,NaN,7,2024,Tarrant County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant County
19,1547,6/30/2024,7A,(1547) Elite Performance Chiropractic,6220 Virginia Parkway,MCKINNEY,TEXAS,75071,First United Bank and Trust Company,4239,None,1400 W Main St,DURANT,OK,74701,1624600.0,1218450.0,2023-01-05,2023,1/30/2023,Preferred Lender Program,Guaranty,9.25,V,300,621310.0,Offices of Chiropractors,NaN,NaN,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 03,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Term,15,NaN,7,2024,Collin County,TEXAS,0305.47,3023,Collin County,33.199333920070615,-96.70156262879846,{'latitute': '33.199333920070615'},(1547) Elite Performance Chiropractic,"6220 Virginia Parkway, MCKINNEY, TEXAS, 75071",Match,Exact,Collin County
30,2354,6/30/2024,7A,(2354) Event Finders LLC,5301 Alpha Road #134,DALLAS,TEXAS,75240,Encore Bank,34562,None,1801 Rahling Rd,LITTLE ROCK,AR,72223,150000.0,75000.0,2023-01-06,2023,2/3/2023,SBA Express,FA$TRK (Small Loan Express),9.75,V,120,541990.0,"All Other Professional, Scientific, and Techni...",NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 24,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Revolver,6,NaN,7,2024,Dallas County,TEXAS,0136.26,2006,Dallas County,32.932959530641426,-96.82165419461336,{'latitute': '32.932959530641426'},(2354) Event Finders LLC,"5301 Alpha Road #134, DALLAS, TEXAS, 75240",Match,Exact,Dallas County


# **Joins**

## hmda_panel to hmda_lar

In [35]:
hmda_dict_panel_df_sub = hmda_dict_panel_df[['lei','respondent_name']]

hmda_lar_panel_df = pd.merge(\
    hmda_dict_lar_df, hmda_dict_panel_df_sub,\
    left_on = ['lei'], right_on = ['lei'], how = 'left')\
.rename(columns = {'respondent_name':'institution_name'})

hmda_lar_panel_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,census_tract_full,institution_name
0,2023,549300JOT0D4J0SZIK67,19124,TEXAS,Dallas County,0190.24,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Freddie Mac,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,195000,NaN,5.875,NaN,Not a high-cost mortgage,6600.9,NaN,3880.0,665.0,NaN,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,455000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,8888,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,4164,58.29,105600,110.56,1207,1498,0,NaN,48113019024.0,THE MONEY SOURCE INC.
1,2023,549300JOT0D4J0SZIK67,23104,TEXAS,Tarrant County,1113.19,NaN,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Ginnie Mae,Preapproval not requested,Veterans Affairs guaranteed (VA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,745000,NaN,4.75,NaN,Not a high-cost mortgage,12082.02,NaN,5975.0,4980.0,265.45,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,745000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not

In [36]:
# below shows so we do not have a matching bank name for every lei 
print('lar',hmda_dict_lar_df['lei'].nunique(),'\nmerged panel and lar',\
      hmda_lar_panel_df['institution_name'].nunique())

lar 1090 
merged panel and lar 1079


In [37]:
# BANK NAME INTO ORIGINAL LAR DF 

hmda_dict_lar_df = pd.merge(\
    hmda_dict_lar_df, hmda_dict_panel_df_sub,\
    left_on = ['lei'], right_on = ['lei'], how = 'left')\
.rename(columns = {'respondent_name':'institution_name'})

hmda_dict_lar_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,census_tract_full,institution_name
0,2023,549300JOT0D4J0SZIK67,19124,TEXAS,Dallas County,0190.24,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Freddie Mac,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,195000,NaN,5.875,NaN,Not a high-cost mortgage,6600.9,NaN,3880.0,665.0,NaN,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,455000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,8888,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,4164,58.29,105600,110.56,1207,1498,0,NaN,48113019024.0,THE MONEY SOURCE INC.
1,2023,549300JOT0D4J0SZIK67,23104,TEXAS,Tarrant County,1113.19,NaN,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Ginnie Mae,Preapproval not requested,Veterans Affairs guaranteed (VA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,745000,NaN,4.75,NaN,Not a high-cost mortgage,12082.02,NaN,5975.0,4980.0,265.45,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,745000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not

# **Add Race_Ethnicity field to HDMA data** 

if the race is in (Asian, asian indian, other asian, vietnamese, filipino, Korean, Japanese) then asian 

if hmda_dict_lar_df['applicant_race_1'] == "White" and "Hispanic or Latino" in hmda_dict_lar_df['applicant_ethnicity_1'] then hmda_dict_lar_df['applicant_race_1'] +' '+ hmda_dict_lar_df['applicant_ethnicity_1']

if race == "Black or African American" then "Black or African American"

else "Other"

In [38]:
# old: applicant_race_1, applicant_ethnicity_1

race_field = 'derived_race'
eth_field = 'derived_ethnicity'

hmda_dict_lar_df['Race_Ethnicity'] = np.where(\
    (hmda_dict_lar_df[race_field] == "Black or African American"),"Black or African American",\
    np.where(\
        ((hmda_dict_lar_df[eth_field] == "Hispanic or Latino")|\
        (hmda_dict_lar_df[eth_field] == "Other Hispanic or Latino")),"Hispanic or Latino",\
        np.where(\
            (hmda_dict_lar_df['applicant_race_1'] == "White"), "White not Hispanic or Latino",\
            np.where(\
                ((hmda_dict_lar_df[race_field] == "Asian")|\
                 (hmda_dict_lar_df[race_field] == "Asian Indian")|\
                 (hmda_dict_lar_df[race_field] == "Other Asian")|\
                 (hmda_dict_lar_df[race_field] == "Vietnamese")|\
                 (hmda_dict_lar_df[race_field] == "Filipino")|\
                 (hmda_dict_lar_df[race_field] == "Korean")|\
                 (hmda_dict_lar_df[race_field] == "Japanese")), "Asian", "Other"
            )
        )
         
    )
        )

hmda_dict_lar_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,census_tract_full,institution_name,Race_Ethnicity
0,2023,549300JOT0D4J0SZIK67,19124,TEXAS,Dallas County,0190.24,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Freddie Mac,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,195000,NaN,5.875,NaN,Not a high-cost mortgage,6600.9,NaN,3880.0,665.0,NaN,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,455000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,8888,NaN,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,4164,58.29,105600,110.56,1207,1498,0,NaN,48113019024.0,THE MONEY SOURCE INC.,Other
1,2023,549300JOT0D4J0SZIK67,23104,TEXAS,Tarrant County,1113.19,NaN,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,Purchased loan,Ginnie Mae,Preapproval not requested,Veterans Affairs guaranteed (VA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,745000,NaN,4.75,NaN,Not a high-cost mortgage,12082.02,NaN,5975.0,4980.0,265.45,360,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,745000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,NaN,Unknown,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,Not applicab

In [39]:
hmda_dict_lar_df['Race_Ethnicity'].value_counts()

White not Hispanic or Latino    57311
Other                           53637
Hispanic or Latino              29729
Asian                           24642
Black or African American       19529
Name: Race_Ethnicity, dtype: int64

# **Summary Tables** 

## Branch-level details

In [40]:
fdic_branch_details = fdic_locations_df.copy()[['Institution Name','Office Name','Branch Address', 'Branch City',\
                   'Branch State', 'Branch Zip Code', '_tract', 'Branch County']]

fdic_branch_details.rename(columns={'Institution Name': 'institution_name'}, inplace=True) 
fdic_branch_details = clean_inst_names(fdic_branch_details)

fdic_branch_details.head(5)

,institution_name,Office Name,Branch Address,Branch City,Branch State,Branch Zip Code,_tract,Branch County
0,INDEPENDENT BANK,McKinney Branch,1650 N Central Expy,Mckinney,Texas,75070,0307.01,Collin County
1,ORIGIN BANK,CAMP BOWIE BANKING CENTER,5800 Camp Bowie Blvd,Fort Worth,Texas,76107,0000NAN,Tarrant County
2,TEXAS EXCHANGE BANK,DOWNTOWN FORT WORTH-SUNDANCE SQUARE,301 Commerce St,Fort Worth,Texas,76102,1233.02,Tarrant County
3,BANK OZK,SOUTHLAKE BRANCH,600 W Southlake Blvd,Southlake,Texas,76092,1139.06,Tarrant County
4,SIMMONS BANK,Fort Worth 7th Street Branch,2200 W 7th St,Fort Worth,Texas,76107,1020.00,Tarrant County


In [41]:
# Use the format_census_tract function to format the census tract column
fdic_branch_details['tract_fmt'] = fdic_branch_details['_tract'].apply(format_census_tract)
fdic_branch_details['tract_fmt_no_decimal'] = fdic_branch_details['_tract'].apply(format_census_tract, remove_decimal=True)
fdic_branch_details.head()


,institution_name,Office Name,Branch Address,Branch City,Branch State,Branch Zip Code,_tract,Branch County,tract_fmt,tract_fmt_no_decimal
0,INDEPENDENT BANK,McKinney Branch,1650 N Central Expy,Mckinney,Texas,75070,0307.01,Collin County,0307.01,030701
1,ORIGIN BANK,CAMP BOWIE BANKING CENTER,5800 Camp Bowie Blvd,Fort Worth,Texas,76107,0000NAN,Tarrant County,0000NAN,0000NAN
2,TEXAS EXCHANGE BANK,DOWNTOWN FORT WORTH-SUNDANCE SQUARE,301 Commerce St,Fort Worth,Texas,76102,1233.02,Tarrant County,1233.02,123302
3,BANK OZK,SOUTHLAKE BRANCH,600 W Southlake Blvd,Southlake,Texas,76092,1139.06,Tarrant County,1139.06,113906
4,SIMMONS BANK,Fort Worth 7th Street Branch,2200 W 7th St,Fort Worth,Texas,76107,1020.00,Tarrant County,1020.00,102000


In [42]:
# Export FDIC Branch Details to Excel
out_file_path = os.path.join(excel_exports_folder, "fdic_branch_details.xlsx")
export_to_excel(fdic_branch_details, out_file_path, sheet_name="fdic_branch_details", startcol=0, startrow=0)

# Alternative if it throws IllegalCharacterError
# !pip install xlsxwriter
# fdic_branch_details.to_excel(out_file_path, engine='xlsxwriter')

Exporting dataframe to excel-exports/2023/fdic_branch_details.xlsx


## **Census tract-level** summaries

In [28]:
# Get race counts by census tract
# Doesn't include Hispanic or Latino ethnicity consideration

# ffiec_data columns: 

# Asian - Total population Asian
# Asian - Total population White and Asian
# Asian - Total population American Indian or Alaska native
# Asian - Total population White and Native Hawaiian or other Pacific Islander
# Asian - Total population Native Hawaiian or other Pacific Islander
# Asian - Total population White and American Indian or Alaska Native

# Black - Total population Black/African American
# Black - Total population White and Black
# Black - Total population Black and American Indian or Alaska Native

# Other - instead of using `Total population some other race`, compute as `Total population` - `Total Population Asian` - `Total Population Black` - `Total Population White`

# White - Total population White
# White - Total population White and some other race

# ffiec_data['Total Population Asian'] = ffiec_data['Total population Asian'] + ffiec_data['Total population White and Asian'] + \
#     ffiec_data['Total population American Indian or Alaska native'] + \
#     ffiec_data['Total population White and Native Hawaiian or other Pacific Islander'] + \
#     ffiec_data['Total population Native Hawaiian or other Pacific Islander']

# ffiec_data['Total Population Black'] = ffiec_data['Total population Black/African American'] + \
#     ffiec_data['Total population White and Black'] + ffiec_data['Total population Black and American Indian or Alaska Native']

# ffiec_data['Total Population White'] = ffiec_data['Total population White'] + ffiec_data['Total population White and some other race']

# ffiec_data['Total population'] = ffiec_data['Total population by race']

# ffiec_data['Total Population Other'] = ffiec_data['Total population'] - ffiec_data['Total Population Asian'] - \
#     ffiec_data['Total Population Black'] - ffiec_data['Total Population White']

In [29]:
# ffiec_intermed_df = ffiec_data.groupby(['County','Census tract'], as_index = False)\
# [['Total persons', 'Total families','Total households','Tract Income Indicator', 
#   'Total Population Asian', 'Total Population Black', 'Total Population Other', 'Total Population White', 'Total population']].sum()

# ffiec_intermed_df.head(5)

In [30]:
# # Compute percentage of each racial group, multiply by 100 and round to 2 decimal places
# ffiec_intermed_df['Percent Asian'] = (ffiec_intermed_df['Total Population Asian'] / ffiec_intermed_df['Total population']) * 100
# ffiec_intermed_df['Percent Black or African American'] = (ffiec_intermed_df['Total Population Black'] / ffiec_intermed_df['Total population']) * 100
# ffiec_intermed_df['Percent Hispanic or Latino'] = (ffiec_intermed_df['Total Population Other'] / ffiec_intermed_df['Total population']) * 100
# ffiec_intermed_df['Percent White not Hispanic or Latino'] = (ffiec_intermed_df['Total Population White'] / ffiec_intermed_df['Total population']) * 100

# # Round each percentage to 2 decimal places
# ffiec_intermed_df['Percent Asian'] = ffiec_intermed_df['Percent Asian'].round(2)
# ffiec_intermed_df['Percent Black or African American'] = ffiec_intermed_df['Percent Black or African American'].round(2)
# ffiec_intermed_df['Percent Hispanic or Latino'] = ffiec_intermed_df['Percent Hispanic or Latino'].round(2)
# ffiec_intermed_df['Percent White not Hispanic or Latino'] = ffiec_intermed_df['Percent White not Hispanic or Latino'].round(2)

# # Compute the sum of the racial percentages without using sum_eth
# ffiec_intermed_df['Sum of Racial Percentages'] = ffiec_intermed_df['Percent Asian'] + ffiec_intermed_df['Percent Black or African American'] + \
#     ffiec_intermed_df['Percent Hispanic or Latino'] + ffiec_intermed_df['Percent White not Hispanic or Latino']

# ffiec_intermed_df.head(5)


In [43]:
# hmda originated
hmda_loan_count = hmda_dict_lar_df[hmda_dict_lar_df['action_taken'] == 'Loan originated']\
.groupby(['county_code','census_tract'],as_index = False)\
[['lei']].count().rename(columns = {'lei':'number_of_hmda_loans'})

hmda_loan_count.head(5)

,county_code,census_tract,number_of_hmda_loans
0,Collin County,0301.01,67
1,Collin County,0301.02,79
2,Collin County,0302.01,708
3,Collin County,0302.02,1250
4,Collin County,0302.04,515


In [44]:
# HMDA loan counts by race / ethnicity within each county, census tract, and institution

hmda_race_sex_count_df = hmda_dict_lar_df[hmda_dict_lar_df['action_taken'] == 'Loan originated']\
.groupby(['county_code', 'census_tract', 'institution_name', 'derived_sex'])\
[['Race_Ethnicity']]\
.value_counts(normalize = False)\
.unstack()\
.reset_index()

# Standardize institution names
hmda_race_sex_count_df = clean_inst_names(hmda_race_sex_count_df)

# Census tract formatted without decimal
hmda_race_sex_count_df.insert(2, 'Census_Tract_no_decimal', hmda_race_sex_count_df['census_tract'].str.replace('.',''))

# Replace NA counts with zeroes
values = {'Asian': 0.0, 
          'Black or African American': 0.0,
          'Hispanic or Latino': 0.0, 
          'Other': 0.0, 
          'White not Hispanic or Latino': 0.0}
hmda_race_sex_count_df = hmda_race_sex_count_df.fillna(values)

# Sum across race/ethnicity
hmda_race_sex_count_df['Total'] = hmda_race_sex_count_df.apply(sum_eth, axis=1)

hmda_race_sex_count_df.head(5)

/var/folders/d0/c58w49297vd0qvd2sb51sdzm0000gn/T/ipykernel_5409/3259295417.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  hmda_race_sex_count_df.insert(2, 'Census_Tract_no_decimal', hmda_race_sex_count_df['census_tract'].str.replace('.',''))


Race_Ethnicity,county_code,census_tract,Census_Tract_no_decimal,institution_name,derived_sex,Asian,Black or African American,Hispanic or Latino,Other,White not Hispanic or Latino,Total
0,Collin County,0301.01,030101,21ST MORTGAGE CORPORATION,Joint,0.0,0.0,0.0,0.0,2.0,2.0
1,Collin County,0301.01,030101,A&D MORTGAGE LLC,Male,0.0,0.0,0.0,0.0,1.0,1.0
2,Collin County,0301.01,030101,AMERICAN FINANCIAL NETWORK INC,Joint,0.0,0.0,1.0,0.0,0.0,1.0
3,Collin County,0301.01,030101,AMERICAN PACIFIC MORTGAGE CORPORATION,Female,0.0,0.0,1.0,0.0,0.0,1.0
4,Collin County,0301.01,030101,ARK-LA-TEX FINANCIAL SERVICES LLC,Female,0.0,0.0,0.0,0.0,1.0,1.0


In [45]:
# Export HMDA race counts to Excel
out_file_path = os.path.join(excel_exports_folder, "hmda_race_sex_count_by_inst.xlsx")
export_to_excel(hmda_race_sex_count_df, out_file_path, sheet_name="hmda_race_sex_count_df", startcol=0, startrow=0)

Exporting dataframe to excel-exports/2023/hmda_race_sex_count_by_inst.xlsx


# FFIEC data summaries

In [46]:
base_ffiec_columns = ['County', 'Census tract', 'Total persons', 'Total families', 'Total households', 'Tract Income Indicator']

ffiec_race_cnt_cols = ['Total Population Asian', 'Total Population Black', 'Total Population Other', 'Total Population White', 
'Total population']

ffiec_race_pct_cols = ['Percent Asian', 'Percent Black or African American', 'Percent Hispanic or Latino', 'Percent White not Hispanic or Latino']



In [47]:
# Show the last 20 columns of the FFIEC data
ffiec_data.iloc[:, -20:].head(5)

,"Total owner-occupied units $50,000-$59,999","Total owner-occupied units $60,000-$69,999","Total owner-occupied units $70,000-$79,999","Total owner-occupied units $80,000-$89,999","Total owner-occupied units $90,000-$99,999","Total owner-occupied units $100,000-$124,999","Total owner-occupied units $125,000-$149,999","Total owner-occupied units $150,000-$174,999","Total owner-occupied units $175,000-$199,999","Total owner-occupied units $200,000-$249,999","Total owner-occupied units $250,000-$299,999","Total owner-occupied units $300,000-$399,999","Total owner-occupied units $400,000-$499,999","Total owner-occupied units $500,000-$749,999","Total owner-occupied units $750,000-$999,999","Total owner-occupied units $1,000,000 to $1,499,999","Total owner-occupied units $1,500,000 to $1,999,999","Total owner-occupied units $2,000,000 or more",Median value in dollars for all owner-occupied housing units,Tract Income Indicator
72753,2.0,10.0,59.0,0.0,9.0,51.0,87.0,41.0,149.0,58.0,16.0,64.0,54.0,30.0,1.0,0.0,0.0,0.0,183600.0,Middle Income
72754,19.0,48.0,42.0,28.0,3.0,88.0,28.0,43.0,60.0,28.0,39.0,155.0,46.0,100.0,13.0,13.0,0.0,0.0,198500.0,Low or Moderate Income
72755,5.0,0.0,15.0,25.0,19.0,24.0,20.0,27.0,24.0,107.0,129.0,165.0,142.0,112.0,66.0,5.0,7.0,0.0,324200.0,Middle Income
72756,0.0,7.0,0.0,18.0,17.0,14.0,0.0,24.0,15.0,37.0,120.0,310.0,197.0,139.0,54.0,24.0,5.0,2.0,366900.0,Upper Income
72757,0.0,0.0,0.0,0.0,15.0,6.0,60.0,200.0,215.0,646.0,183.0,142.0,0.0,0.0,0.0,0.0,0.0,0.0,217200.0,Middle Income


## FFIEC Version 1: Using race fields only (no Hispanic ethnicity consideration)

In [48]:
# Compute the sum of the racial populations
ffiec_data['Total Population Asian'] = ffiec_data['Total population Asian'] + ffiec_data['Total population White and Asian'] + \
    ffiec_data['Total population American Indian or Alaska native'] + \
    ffiec_data['Total population White and Native Hawaiian or other Pacific Islander'] + \
    ffiec_data['Total population Native Hawaiian or other Pacific Islander']

ffiec_data['Total Population Black'] = ffiec_data['Total population Black/African American'] + \
    ffiec_data['Total population White and Black'] + ffiec_data['Total population Black and American Indian or Alaska Native']

ffiec_data['Total Population White'] = ffiec_data['Total population White'] + ffiec_data['Total population White and some other race']

ffiec_data['Total population'] = ffiec_data['Total population by race']

ffiec_data['Total Population Other'] = ffiec_data['Total population'] - ffiec_data['Total Population Asian'] - \
    ffiec_data['Total Population Black'] - ffiec_data['Total Population White']

In [60]:
# Selected columns from ffiec_data for joins
ffiec_subset = ffiec_data[base_ffiec_columns + ffiec_race_cnt_cols].copy()

# Compute percentages for each racial group
ffiec_subset['Percent Asian'] = (ffiec_subset['Total Population Asian'] / ffiec_subset['Total population']) * 100
ffiec_subset['Percent Black or African American'] = (ffiec_subset['Total Population Black'] / ffiec_subset['Total population']) * 100
ffiec_subset['Percent White'] = (ffiec_subset['Total Population White'] / ffiec_subset['Total population']) * 100
ffiec_subset['Percent Other'] = (ffiec_subset['Total Population Other'] / ffiec_subset['Total population']) * 100

# Round each percentage to 2 decimal places
ffiec_subset['Percent Asian'] = ffiec_subset['Percent Asian'].round(2)
ffiec_subset['Percent Black or African American'] = ffiec_subset['Percent Black or African American'].round(2)
ffiec_subset['Percent White'] = ffiec_subset['Percent White'].round(2)
ffiec_subset['Percent Other'] = ffiec_subset['Percent Other'].round(2)

# Join to cra_income_group_lookup
ffiec_subset = pd.merge(ffiec_subset, cra_income_group_lookup, how='left', 
left_on=['County', 'Census tract'],
right_on=['County', 'Census Tract'])


ffiec_subset.head(5)

,County,Census tract,Total persons,Total families,Total households,Tract Income Indicator,Total Population Asian,Total Population Black,Total Population Other,Total Population White,Total population,Percent Asian,Percent Black or African American,Percent White,Percent Other,Census Tract,CRA Tract Income Group
0,Collin County,0301.01,3595.0,596.0,750.0,Middle Income,101.0,59.0,467.0,2968.0,3595.0,2.81,1.64,82.56,12.99,0301.01,80% to 90% of MFI
1,Collin County,0301.02,4282.0,831.0,1011.0,Low or Moderate Income,117.0,61.0,556.0,3548.0,4282.0,2.73,1.42,82.86,12.98,0301.02,70% to 80% of MFI
2,Collin County,0302.01,6175.0,1024.0,1154.0,Middle Income,343.0,668.0,517.0,4647.0,6175.0,5.55,10.82,75.26,8.37,0302.01,100% to 110% of MFI
3,Collin County,0302.02,6345.0,983.0,1186.0,Upper Income,416.0,585.0,411.0,4933.0,6345.0,6.56,9.22,77.75,6.48,0302.02,> 120% of MFI
4,Collin County,0302.04,7554.0,2019.0,2354.0,Middle Income,300.0,1044.0,822.0,5388.0,7554.0,3.97,13.82,71.33,10.88,0302.04,110% to 120% of MFI


In [61]:
# Export FFIEC subset to Excel
out_file_path = os.path.join(excel_exports_folder, "ffiec_cra_pop_and_income_by_tract.xlsx")
export_to_excel(ffiec_subset, out_file_path, sheet_name="ffiec_cra_pop_and_income_by_tr", startcol=0, startrow=0)

Exporting dataframe to excel-exports/2023/ffiec_cra_pop_and_income_by_tract.xlsx


## FFIEC Version 2: Using Hispanic and non-Hispanic by Race fields [Preferred]

In [62]:
ffiec_data_summary = ffiec_data.copy()
# hispanic
ffiec_data_summary['Total Population Hispanic']\
= ffiec_data_summary['Total population Hispanic or Latino by race']

# white
ffiec_data_summary['Total Population White']\
= ffiec_data_summary['Total population non-Hispanic White']

# black
ffiec_data_summary['Total Population Black']\
= ffiec_data_summary\
['Total population non-Hispanic White and Black'] +\
ffiec_data_summary\
['Total population non-Hispanic Black/African American'] +\
ffiec_data_summary\
['Total population non-Hispanic Black and American Indian or Alaska native']

# asian
ffiec_data_summary['Total Population Asian']\
= ffiec_data_summary\
['Total population non-Hispanic Asian']+\
    ffiec_data_summary\
['Total population non-Hispanic native Hawaiian or other Pacific Islander']+\
    ffiec_data_summary\
['Total population non-Hispanic White and Asian']+\
    ffiec_data_summary\
['Total population non-Hispanic White and Native Hawaiian or other Pacific Islander']

# total population
ffiec_data_summary['Total Population'] = ffiec_data_summary['Total population Hispanic and Non-Hispanic']

# other
ffiec_data_summary['Total Population Other'] = ffiec_data_summary['Total Population'] - \
    ffiec_data_summary['Total Population Hispanic'] - ffiec_data_summary['Total Population White'] - \
        ffiec_data_summary['Total Population Black'] - ffiec_data_summary['Total Population Asian']


ffiec_data_summary['Calc Total'] = ffiec_data_summary['Total Population Hispanic'] + ffiec_data_summary['Total Population White'] + \
    ffiec_data_summary['Total Population Black'] + ffiec_data_summary['Total Population Asian'] + ffiec_data_summary['Total Population Other']

base_ffiec_columns = ['County', 'Census tract', 'Total persons', 'Total families', 'Total households', 'Tract Income Indicator']
ffiec_race_cnt_cols = ['Total Population Asian', 'Total Population Black', 'Total Population Hispanic', 'Total Population Other', 
                       'Total Population White', 'Total Population']
ffiec_data_summary = ffiec_data_summary[base_ffiec_columns + ffiec_race_cnt_cols + ['Calc Total']]
    
ffiec_data_summary.head(5)

,County,Census tract,Total persons,Total families,Total households,Tract Income Indicator,Total Population Asian,Total Population Black,Total Population Hispanic,Total Population Other,Total Population White,Total Population,Calc Total
72753,Collin County,0301.01,3595.0,596.0,750.0,Middle Income,53.0,56.0,782.0,194.0,2510.0,3595.0,3595.0
72754,Collin County,0301.02,4282.0,831.0,1011.0,Low or Moderate Income,49.0,57.0,832.0,205.0,3139.0,4282.0,4282.0
72755,Collin County,0302.01,6175.0,1024.0,1154.0,Middle Income,255.0,648.0,905.0,293.0,4074.0,6175.0,6175.0
72756,Collin County,0302.02,6345.0,983.0,1186.0,Upper Income,366.0,570.0,850.0,242.0,4317.0,6345.0,6345.0
72757,Collin County,0302.04,7554.0,2019.0,2354.0,Middle Income,206.0,1013.0,1705.0,298.0,4332.0,7554.0,7554.0


In [63]:
# Sanity check... do the totals match?
ffiec_data_summary[['Total Population','Calc Total']].head(5)

,Total Population,Calc Total
72753,3595.0,3595.0
72754,4282.0,4282.0
72755,6175.0,6175.0
72756,6345.0,6345.0
72757,7554.0,7554.0


In [64]:
# subset for values of interest
ffiec_data_summary = ffiec_data_summary[[
    'County',
    'Census tract', 'Tract Income Indicator',
    'Total persons', 'Total families', 'Total households',
    'Total Population Hispanic',
    'Total Population White',
    'Total Population Black',
    'Total Population Asian',
    'Total Population Other',
    'Total Population'
    ]]

In [65]:
ffiec_data_summary['Census_Tract_no_decimal'] = ffiec_data_summary['Census tract'].str.replace('.','')

# Create a new column in ffiec_data_summary called 'data_source' and set it to 'FFIEC'
# Use a method that avoids SettingWithCopyWarning
#ffiec_data_summary = ffiec_data_summary.assign(data_source='FFIEC')
ffiec_data_summary.head(5)

/var/folders/d0/c58w49297vd0qvd2sb51sdzm0000gn/T/ipykernel_5409/4063391160.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ffiec_data_summary['Census_Tract_no_decimal'] = ffiec_data_summary['Census tract'].str.replace('.','')


,County,Census tract,Tract Income Indicator,Total persons,Total families,Total households,Total Population Hispanic,Total Population White,Total Population Black,Total Population Asian,Total Population Other,Total Population,Census_Tract_no_decimal
72753,Collin County,0301.01,Middle Income,3595.0,596.0,750.0,782.0,2510.0,56.0,53.0,194.0,3595.0,030101
72754,Collin County,0301.02,Low or Moderate Income,4282.0,831.0,1011.0,832.0,3139.0,57.0,49.0,205.0,4282.0,030102
72755,Collin County,0302.01,Middle Income,6175.0,1024.0,1154.0,905.0,4074.0,648.0,255.0,293.0,6175.0,030201
72756,Collin County,0302.02,Upper Income,6345.0,983.0,1186.0,850.0,4317.0,570.0,366.0,242.0,6345.0,030202
72757,Collin County,0302.04,Middle Income,7554.0,2019.0,2354.0,1705.0,4332.0,1013.0,206.0,298.0,7554.0,030204


In [66]:
# Merge ffiec_data_summary with cra_income_group_lookup
ffiec_data_summary = pd.merge(ffiec_data_summary, cra_income_group_lookup, 
                              left_on=['County', 'Census tract'], 
                              right_on=['County', 'Census Tract'], how='left') 

In [67]:
# ffiec_race_pct_cols = ['Percent Asian', 'Percent Black', 'Percent Hispanic', 'Percent White', 'Percent Other']

# Compute Percentage of each racial group, multiply by 100 and round to 2 decimal places
ffiec_data_summary['Percent Hispanic'] = (ffiec_data_summary['Total Population Hispanic'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent White'] = (ffiec_data_summary['Total Population White'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent Black'] = (ffiec_data_summary['Total Population Black'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent Asian'] = (ffiec_data_summary['Total Population Asian'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent Other'] = (ffiec_data_summary['Total Population Other'] / ffiec_data_summary['Total Population']) * 100

# Round each percentage to 2 decimal places
ffiec_data_summary['Percent Hispanic'] = ffiec_data_summary['Percent Hispanic'].round(2)
ffiec_data_summary['Percent White'] = ffiec_data_summary['Percent White'].round(2)
ffiec_data_summary['Percent Black'] = ffiec_data_summary['Percent Black'].round(2)
ffiec_data_summary['Percent Asian'] = ffiec_data_summary['Percent Asian'].round(2)
ffiec_data_summary['Percent Other'] = ffiec_data_summary['Percent Other'].round(2)

ffiec_data_summary.head(5)


,County,Census tract,Tract Income Indicator,Total persons,Total families,Total households,Total Population Hispanic,Total Population White,Total Population Black,Total Population Asian,Total Population Other,Total Population,Census_Tract_no_decimal,Census Tract,CRA Tract Income Group,Percent Hispanic,Percent White,Percent Black,Percent Asian,Percent Other
0,Collin County,0301.01,Middle Income,3595.0,596.0,750.0,782.0,2510.0,56.0,53.0,194.0,3595.0,030101,0301.01,80% to 90% of MFI,21.75,69.82,1.56,1.47,5.40
1,Collin County,0301.02,Low or Moderate Income,4282.0,831.0,1011.0,832.0,3139.0,57.0,49.0,205.0,4282.0,030102,0301.02,70% to 80% of MFI,19.43,73.31,1.33,1.14,4.79
2,Collin County,0302.01,Middle Income,6175.0,1024.0,1154.0,905.0,4074.0,648.0,255.0,293.0,6175.0,030201,0302.01,100% to 110% of MFI,14.66,65.98,10.49,4.13,4.74
3,Collin County,0302.02,Upper Income,6345.0,983.0,1186.0,850.0,4317.0,570.0,366.0,242.0,6345.0,030202,0302.02,> 120% of MFI,13.40,68.04,8.98,5.77,3.81
4,Collin County,0302.04,Middle Income,7554.0,2019.0,2354.0,1705.0,4332.0,1013.0,206.0,298.0,7554.0,030204,0302.04,110% to 120% of MFI,22.57,57.35,13.41,2.73,3.94


In [68]:
# Export FFIEC data summary to Excel
out_file_path = os.path.join(excel_exports_folder, "ffiec_data_summary_wide.xlsx")
export_to_excel(ffiec_data_summary, out_file_path, sheet_name="ffiec_data_summary", startcol=0, startrow=0)

Exporting dataframe to excel-exports/2023/ffiec_data_summary_wide.xlsx


# Intermediate dataframes for merge

In [69]:
# Use tract income lookup as the final dataframe for FFIEC data
ffiec_intermed_df = ffiec_tract_income_lookup.copy()

ffiec_intermed_df.head(5)

,index,State,County,Census tract,Tract Income Indicator,Census_Tract_No_Decimal
0,72753,TEXAS,Collin County,0301.01,Middle Income,030101
1,72754,TEXAS,Collin County,0301.02,Low or Moderate Income,030102
2,72755,TEXAS,Collin County,0302.01,Middle Income,030201
3,72756,TEXAS,Collin County,0302.02,Upper Income,030202
4,72757,TEXAS,Collin County,0302.04,Middle Income,030204


## Interm calc 1: HMDA loan originations

In [70]:
# hmda originated
hmda_loans_orig = hmda_dict_lar_df[hmda_dict_lar_df['action_taken'] == 'Loan originated'].copy()
hmda_loans_orig = clean_inst_names(hmda_loans_orig)


hmda_intermed_df = hmda_loans_orig\
.groupby(['county_code','census_tract', 'institution_name'],as_index = False)\
[['lei']].count().rename(columns = {'lei':'number_of_hmda_loans'})

hmda_intermed_df.head(3)

,county_code,census_tract,institution_name,number_of_hmda_loans
0,Collin County,0301.01,21ST MORTGAGE CORPORATION,2
1,Collin County,0301.01,A&D MORTGAGE LLC,1
2,Collin County,0301.01,AMERICAN AIRLINES FCU,1


In [71]:
# Write a function to Group ahl_funding_loans by Race_Ethnicity and count the number of loans for a given institution name
def count_loans_by_race(institution_name):
    print(f"\nCount loans by Race/Ethnicity for {institution_name}\n")
    return hmda_loans_orig[hmda_loans_orig['institution_name'] == institution_name]\
    .groupby('Race_Ethnicity').size()

count_loans_by_race('AHL FUNDING')


Count loans by Race/Ethnicity for AHL FUNDING



Race_Ethnicity
Asian                           1
Black or African American       1
Hispanic or Latino              1
Other                           1
White not Hispanic or Latino    2
dtype: int64

In [72]:
count_loans_by_race('AAXY LLC')


Count loans by Race/Ethnicity for AAXY LLC



Race_Ethnicity
Asian    12
dtype: int64

In [73]:
count_loans_by_race('ALL CREDIT CONSIDERED MORTGAGE INC')


Count loans by Race/Ethnicity for ALL CREDIT CONSIDERED MORTGAGE INC



Race_Ethnicity
Asian                            6
Hispanic or Latino              16
Other                            4
White not Hispanic or Latino     4
dtype: int64

## Interm calc 2: SBA

TO DO: Run SBA data through Google Sheets Geocode to get census tracts?

In [81]:
sba_data.head(1)

,index,asofdate,program,Borrower name,Borrower street address,Borrower city,Borrower state,Borrower zip code,bankname,bankfdicnumber,bankncuanumber,bankstreet,bankcity,bankstate,bankzip,Total loan amount,sbaguaranteedapproval,approvaldate,approvalfiscalyear,firstdisbursementdate,deliverymethod,subprogram,initialinterestrate,fixedorvariableinterestind,terminmonths,naicscode,naicsdescription,franchisecode,franchisename,projectcounty,projectstate,sbadistrictoffice,congressionaldistrict,businesstype,businessage,loanstatus,paidinfulldate,chargeoffdate,grosschargeoffamount,revolverstatus,jobssupported,SOLDSECMRTIND,catalog_month,catalog_year,County_from_Zipcode,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,borrower identifier,borrower street address,Match result,Match type,county_final
4,480,6/30/2024,7A,(480) Spartan 6S Solutions LLC,9800 Hillwood Pkwy. Suite 14,Fort Worth,TEXAS,76177,"Colonial Savings, F.A.",31361,None,2600 West Frwy,FORT WORTH,TX,76102,500000.0,250000.0,2023-01-03,2023,1/3/2023,SBA Express,FA$TRK (Small Loan Express),8.5,V,84,484110.0,"General Freight Trucking, Local",NaN,NaN,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,12.0,CORPORATION,NaN,The status of loans that have been disbursed b...,NaN,NaN,0,Revolver,141,NaN,7,2024,Tarrant County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant County


In [83]:
# Find the first 5 rows where _tract = '0000NAN'. Select the columns borrower address, borrower city, borrower state, borrower zip code, and _tract

sba_data_subset = sba_data[['Borrower name', 'Borrower street address', 'Borrower city', 'Borrower state', 'Borrower zip code', '_tract']]
sba_data_subset[sba_data_subset['_tract'] == '0000NAN'].head(5)

,Borrower name,Borrower street address,Borrower city,Borrower state,Borrower zip code,_tract
4,(480) Spartan 6S Solutions LLC,9800 Hillwood Pkwy. Suite 14,Fort Worth,TEXAS,76177,0000NAN
8,(906) Tameap LLC,9749 N Freeway,Fort Worth,TEXAS,76177,0000NAN
12,(990) Tameap LLC,9749 N Freeway,Fort Worth,TEXAS,76177,0000NAN
63,(4359) Soleil Cutz LLC,800 East Sublett Road,Arlington,TEXAS,76018,0000NAN
77,(5094) Complete Dentistry in Las Coli,5981 HIGH POINT DR,IRVING,TEXAS,75038,0000NAN


In [84]:
sba_data_subset[sba_data_subset['_tract'] != '0000NAN'].head(5)

,Borrower name,Borrower street address,Borrower city,Borrower state,Borrower zip code,_tract
19,(1547) Elite Performance Chiropractic,6220 Virginia Parkway,MCKINNEY,TEXAS,75071,0305.47
29,(2352) 3Kims 22 Inc,311 S ZANG BLVD,DALLAS,TEXAS,75208,0050.00
30,(2354) Event Finders LLC,5301 Alpha Road #134,DALLAS,TEXAS,75240,0136.26
41,(2476) CWKJ LLC,1211 E. Debbie Ln STE 101,Mansfield,TEXAS,76063,1115.51
46,(3604) Transom Remodeling LLC,2007 N Collins Blvd. Ste 501,RICHARDSON,TEXAS,75080,0192.10


In [74]:
sba_clean = clean_inst_names(sba_data)

sba_clean['Census_Tract_no_decimal'] = sba_clean['_tract'].apply(format_census_tract, remove_decimal=True)

sba_clean.head(5)

KeyError: 'institution_name'

In [ ]:
# Export SBA Loan Details to Excel
out_file_path = os.path.join(excel_exports_folder, "sba_loan_details.xlsx")
export_to_excel(sba_clean, out_file_path, sheet_name="sba_loan_details", startcol=0, startrow=0)


In [ ]:
# Calc percentage of rows where _tract = '0000NAN'
sba_clean[sba_clean['_tract'] == '0000NAN'].shape[0] / sba_clean.shape[0]

In [ ]:
sba_intermed_df = sba_clean\
.groupby(['county_final','_tract', 'institution_name'],
         as_index = False)\
[['Borrower name']].count().rename(\
    columns = {'Borrower name':'sba_loan_count'})

sba_intermed_df.head(10)

In [ ]:
# merge hmda grouping to ffiec grouping
ffiec_hmda_intermed_df = pd.merge(
    ffiec_intermed_df,
    hmda_intermed_df,
    left_on = ['County', 'Census tract'],
    right_on = ['county_code','census_tract'],
    how = 'outer')

ffiec_hmda_intermed_df.head(10)

In [ ]:
# Summary of number_of_hmda_loans variable
print(ffiec_hmda_intermed_df['number_of_hmda_loans'].describe())

# How many nulls?
num_nulls = ffiec_hmda_intermed_df['number_of_hmda_loans'].isnull().sum()
print(f"\nNumber of nulls: {num_nulls}")

In [ ]:
# merge sba to ffiec hmda merge
ffiec_hmda_sba_intermed_df = pd.merge(ffiec_hmda_intermed_df,
                                      sba_intermed_df,
                                      left_on = ['County', 'Census tract', 'institution_name'],
                                      right_on = ['county_final','_tract', 'institution_name'],
                                      how = 'outer')

ffiec_hmda_sba_intermed_df.head(3)

In [ ]:
ffiec_hmda_sba_intermed_df['sba_loan_count'].value_counts()

In [ ]:
# Export merged FFIEC, HMDA, and SBA data to Excel
excel_output_file = os.path.join(excel_exports_folder, "ffiec_hmda_sba_intermed_df.xlsx")
export_to_excel(ffiec_hmda_sba_intermed_df, excel_output_file, sheet_name="ffiec_hmda_sba_intermed_df", startcol=0, startrow=0)

In [56]:
# census_level_hmda_cat_summaries_long LONG FORMAT with institution name included in grouping
census_level_hmda_cat_summaries_long = {}
for category in category_lst: 
    # get value counts for hmda data
    # create a deep copy of the hmda lar data 
    hmda_dict_lar_df_cpy = hmda_dict_lar_df.copy()
    # apply functions to update institution names 
    hmda_dict_lar_df_cpy = clean_inst_names(hmda_dict_lar_df_cpy)
    # filter by originated loans and group by'county_code','census_tract' and'institution_name'
    temp_cat_counts = hmda_dict_lar_df_cpy[hmda_dict_lar_df_cpy['action_taken'] == 'Loan originated']\
    .groupby(['county_code','census_tract','institution_name'])[category].value_counts(dropna = False)\
    .reset_index().fillna(0)
    # create new columns for sum of rows 
    #temp_cat_counts['Total'] = temp_cat_counts.sum(axis = 1, numeric_only = True)
    # bring in ffiec subset
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec subset to hmda value counts df 
    temp_cat_summary = pd.merge(temp_cat_counts,tract_inc_ind, left_on = ['county_code','census_tract'],\
                                right_on = ['County','Census tract'], how = 'left')
    # add  in census tract with no decimal point 
    temp_cat_summary['Census_Tract_no_decimal'] = \
    temp_cat_summary['census_tract'].str.replace('.','')
    # add in hmda column
    temp_cat_summary['data_source'] = 'HMDA'
    # reorder column names
    temp_cat_summary = temp_cat_summary[\
    ['county_code','census_tract','Census_Tract_no_decimal','Tract Income Indicator','data_source'] + 
    temp_cat_counts.drop(['county_code','census_tract'],axis = 1).columns.to_list()]
    census_level_hmda_cat_summaries_long[category] = temp_cat_summary

In [ ]:
census_level_hmda_cat_summaries_long['derived_loan_product_type'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['applicant_age'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['applicant_credit_score_type'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['applicant_sex'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['business_or_commercial_purpose'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['debt_to_income_ratio'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['lien_status'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['loan_purpose'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['loan_type'].head(10)

In [ ]:
census_level_hmda_cat_summaries_long['open_end_line_of_credit'].head(10)

In [67]:
# census_level_sba_cat_summaries_long LONG FORMAT grouping now include BankName->Institution name

census_level_sba_cat_summaries_long = {}
for category in ['DeliveryMethod','CongressionalDistrict','BusinessType','BusinessAge']: 
    # get value counts for sba category columns
    # create deep copy of sba data 
    sba_data_cpy = sba_data.copy()
    # apply functions to update institution names
    sba_data_cpy = clean_inst_names(sba_data_cpy)
    # group sba data by 'county_final','_tract','institution_name'
    temp_cat_counts = sba_data_cpy.groupby(['county_final','_tract','institution_name'])[category].value_counts(dropna = False)\
    .reset_index().fillna(0)
    # create new columns for sum or rows
    #temp_cat_counts['Total'] = temp_cat_counts.sum(axis = 1, numeric_only = True)
    # bring in ffiec subset
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec subset to sba value counts df 
    temp_cat_summary = pd.merge(temp_cat_counts,tract_inc_ind, left_on = ['county_final','_tract'],\
                                right_on = ['County','Census tract'], how = 'left')
    # add  in census tract with no decimal point 
    temp_cat_summary['Census_Tract_no_decimal'] = \
    temp_cat_summary['_tract'].str.replace('.','')

    # add in data source column 
    temp_cat_summary['data_source'] = 'SBA'
    
    # reorder columns names 
    temp_cat_summary = temp_cat_summary[\
    ['county_final','_tract','Census_Tract_no_decimal','Tract Income Indicator','data_source'] + 
    temp_cat_counts.drop(['county_final','_tract'], axis = 1).columns.to_list()].\
    rename(columns = {'county_final':'county_code','_tract':'census_tract'})
    census_level_sba_cat_summaries_long[category] = temp_cat_summary

In [ ]:
census_level_sba_cat_summaries_long['DeliveryMethod'].head(10)

In [ ]:
census_level_sba_cat_summaries_long['CongressionalDistrict'].head(10)

In [ ]:
census_level_sba_cat_summaries_long['BusinessType'].head(10)

In [ ]:
census_level_sba_cat_summaries_long['BusinessAge'].head(10)

In [72]:
# census_level_hmda_num_summaries

census_level_hmda_num_summaries = {}
for num_field in ['loan_amount','interest_rate','origination_charges']:
    # create deep copy of hmda data 
    temp_hmda_df = hmda_dict_lar_df.copy()
    # apply functions to update institution names 
    temp_hmda_df = clean_inst_names(temp_hmda_df)
    # replace "Exempt" with np.nan in numeric fields, group and get median for hmda groupings  
    temp_hmda_df[num_field] = temp_hmda_df[num_field].replace('Exempt',np.nan).apply(float)
    temp_num_counts = temp_hmda_df.\
    groupby(['county_code','census_tract','institution_name'])[num_field].median().reset_index()
    # bring in ffiec subset 
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec data with 5 number summary df 
    temp_num_summary = pd.merge(temp_num_counts,tract_inc_ind, left_on = ['county_code','census_tract'],\
                                 right_on = ['County','Census tract'], how = 'left')
    # add in Census_Tract_no_decimal
    temp_num_summary['Census_Tract_no_decimal'] = \
    temp_num_summary['census_tract'].str.replace('.','')

    # add in data source
    temp_num_summary['data_source'] = 'HMDA'
    
    # add df to dictionary and reorder column names 
    census_level_hmda_num_summaries[num_field] =\
    temp_num_summary[['county_code','census_tract','Census_Tract_no_decimal',\
                      'Tract Income Indicator','data_source',
                      'institution_name',
                     num_field]]\
    .fillna(0).rename(columns = {num_field:'median'})
    

In [ ]:
census_level_hmda_num_summaries.keys()

In [ ]:
census_level_hmda_num_summaries['loan_amount'].head(10)

In [ ]:
census_level_hmda_num_summaries['interest_rate'].head(10)

In [ ]:
census_level_hmda_num_summaries['origination_charges'].head(10)

In [77]:
# census_level_sba_num_summaries

census_level_sba_num_summaries = {}
for num_field in ['Total loan amount',
                  'SBAGuaranteedApproval',
                  'InitialInterestRate',
                  'TermInMonths',
                  'JobsSupported']: 
    # create deep copy of sba data 
    sba_data_cpy = sba_data.copy()
    # apply functions to update institution names
    sba_data_cpy = clean_inst_names(sba_data_cpy)
    # group sba data and get median for groupings 
    temp_num_counts = sba_data_cpy.groupby([
        'county_final',
        '_tract',
        'institution_name'])[num_field].median().reset_index()
    # bring in ffiec subset
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec data with 5 number summary df 
    temp_num_summary = pd.merge(temp_num_counts,tract_inc_ind, left_on = ['county_final','_tract'],\
                                 right_on = ['County','Census tract'], how = 'left')
    # add in Census_Tract_no_decimal
    temp_num_summary['Census_Tract_no_decimal'] = \
    temp_num_summary['_tract'].str.replace('.','')
    # add in data source
    temp_num_summary['data_source'] = 'SBA'
    # add df to dictionary and reorder column names 
    census_level_sba_num_summaries[num_field] =\
    temp_num_summary[['county_final','_tract','Census_Tract_no_decimal','institution_name',\
                      'Tract Income Indicator','data_source',num_field]]\
    .fillna(0).rename(columns = {'county_final':'county_code',
                                 num_field:'median',
                                 '_tract':'census_tract'})

In [ ]:
census_level_sba_num_summaries.keys()

In [ ]:
census_level_sba_num_summaries['Total loan amount'].head(10)

In [ ]:
census_level_sba_num_summaries['SBAGuaranteedApproval'].head(10)

In [ ]:
census_level_sba_num_summaries['InitialInterestRate'].head(10)

In [ ]:
census_level_sba_num_summaries['TermInMonths'].head(10)

In [ ]:
census_level_sba_num_summaries['JobsSupported'].head(10)

## tableau_stack_data_format function for **census level summaries**

In [84]:
lst_of_dcts = [census_level_hmda_cat_summaries_long, census_level_hmda_num_summaries, census_level_sba_cat_summaries_long, census_level_sba_num_summaries]

In [85]:
def tableau_stack_data_format(lst_of_summary_table_dicts):
    summaries_tableau_df = pd.DataFrame()
    for summary_dct in lst_of_summary_table_dicts:
        for summary_table_name in summary_dct.keys():
            temp_df = summary_dct[summary_table_name].copy()
            temp_df = temp_df[['county_code',
                               'census_tract',
                               'institution_name',
                               'Census_Tract_no_decimal',
                               'Tract Income Indicator', # remove this if not grouping by tract indicator
                               'data_source']]
            temp_df['variable_name'] = summary_table_name
            variable_type = ''
            if 'median' in summary_dct[summary_table_name].columns:
                variable_type = 'numeric'
            else:
                variable_type = 'categorical'
            temp_df['variable_type'] = variable_type
            if variable_type == 'numeric':
                temp_df['categorical_level'] = ''
            else:
                temp_df['categorical_level'] = summary_dct[summary_table_name][summary_table_name].copy()
            if variable_type == 'numeric':
                temp_df['value_type'] = 'median'
                temp_df['value'] = summary_dct[summary_table_name]['median'].copy()
            else:
                temp_df['value_type'] = 'count'
                temp_df['value'] = summary_dct[summary_table_name]['count'].copy()
            summaries_tableau_df = pd.concat([summaries_tableau_df,temp_df],ignore_index = True)
            print(summary_table_name,':table added',temp_df.shape[0],'rows added')
    return summaries_tableau_df

In [ ]:
summaries_tableau_df_otp = tableau_stack_data_format(lst_of_dcts)

In [ ]:
summaries_tableau_df_otp.head(10)

In [ ]:
summaries_tableau_df_otp['variable_name'].value_counts()

In [ ]:
summaries_tableau_df_otp['data_source'].value_counts()

# FFIEC : long format

In [90]:
# ffiec_data_summary_long_format = ffiec_data_summary.melt(
#     id_vars = [
#     'State',
#     'County',
#     'Census tract',
#     'Tract Income Indicator'],
#     value_vars = [
#         'Hispanic',
#         'White',
#         'Black',
#         'Asian',
#         'Other'], 
#     var_name = 'Race/Ethnicity',
#     value_name = 'Total Persons in Race Ethnicity Group')

In [91]:
# # adding columns to make help concate ffiec data to hmda, sba long format data
# ffiec_data_summary_long_format = ffiec_data_summary_long_format\
# .rename(columns = {'County':'county_code','Census tract':'census_tract'})
# ffiec_data_summary_long_format['Census_Tract_no_decimal'] = \
# ffiec_data_summary_long_format['census_tract'].str.replace('.','')
# ffiec_data_summary_long_format['data_source'] = 'FFIEC'
# ffiec_data_summary_long_format.head(5)

In [92]:
# add in census tract no decimal field
# ffiec_summary_tableau_df = ffiec_data_summary_long_format[[
#     'county_code',
#     'census_tract',
#     'Census_Tract_no_decimal',
#     'Tract Income Indicator',
#     'data_source']].copy()

In [93]:
# ffiec_summary_tableau_df['variable_name'] = 'Race/Ethnicity'
# ffiec_summary_tableau_df['variable_type'] = 'categorical'
# ffiec_summary_tableau_df['categorical_level'] = ffiec_data_summary_long_format['Race/Ethnicity']
# ffiec_summary_tableau_df['value_type'] = 'count'
# ffiec_summary_tableau_df['value'] = ffiec_data_summary_long_format['Total Persons in Race Ethnicity Group']
# ffiec_summary_tableau_df['institution_name'] = ''
# ffiec_summary_tableau_df = ffiec_summary_tableau_df[[\
#     'county_code',
#     'census_tract',	
#     'institution_name',
#     'Census_Tract_no_decimal',
#     'Tract Income Indicator',
#     'data_source',
#     'variable_name',
#     'variable_type',
#     'categorical_level',
#     'value_type',
#     'value']]
    
# ffiec_summary_tableau_df.head(10)

## **Concatenate summaries**

In [ ]:
# concatenate census tract-level hmda and sba summaries in tableau format with census tract level ffiec summary in tableau format
# tract_hmda_sba_ffiec_summaries_tableau = pd.concat([summaries_tableau_df_otp,ffiec_summary_tableau_df], ignore_index = True)
tract_hmda_sba_ffiec_summaries_tableau = summaries_tableau_df_otp.copy()

tract_hmda_sba_ffiec_summaries_tableau.head(5)

## export to excel workbook

In [ ]:
# export hmda, sba, ffiec tableau formatted large table (9-10 minute run time)
excel_output_file = os.path.join(excel_exports_folder, "banking_dashboard_analysis_export_hmda_sba_ffiec_with_inst.xlsx")
export_to_excel(tract_hmda_sba_ffiec_summaries_tableau, excel_output_file, sheet_name="tract_hmda_sba_ffiec_summaries", startcol=0, startrow=0)

In [ ]:
# export cra 2022 subset data
excel_output_file = os.path.join(excel_exports_folder, "cra_2022_D11_subset.xlsx")
export_to_excel(cra_2022_D11_subset, excel_output_file, sheet_name="cra_2022_D11_subset", startcol=0, startrow=0)

In [ ]:
# export hmda data in long format as multiple sheets(one for each summary)
excel_output_file = os.path.join(excel_exports_folder, "banking_dashboard_analysis_export_hmda.xlsx")

with pd.ExcelWriter(excel_output_file) as writer:
    for hmda_dct in [census_level_hmda_cat_summaries_long, census_level_hmda_num_summaries]:
        for key, value in hmda_dct.items():
            sheet_name = key
            print(f"Exporting results for {sheet_name}")  
            
            # Insert the item label into the A1 cell in the worksheet
            item_label = pd.Series(sheet_name).to_frame()
            item_label.to_excel(writer, sheet_name=sheet_name, startcol=0, startrow=0, index = False, header = False) 
            
            # Insert the dataframe
            value.to_excel(writer, sheet_name=sheet_name, startcol=3, startrow=0) 

In [ ]:
# export sba data in long format as multiple sheets(one for each summary)

excel_output_file = os.path.join(excel_exports_folder, "banking_dashboard_analysis_export_sba.xlsx")

with pd.ExcelWriter(excel_output_file) as writer:
    for sba_dict in [census_level_sba_cat_summaries_long, census_level_sba_num_summaries]:
        for key, value in sba_dict.items():
            sheet_name = key
            print(f"Exporting results for {sheet_name}")  
            
            # Insert the item label into the A1 cell in the worksheet
            item_label = pd.Series(sheet_name).to_frame()
            item_label.to_excel(writer, sheet_name=sheet_name, startcol=0, startrow=0, index = False, header = False) 
            
            # Insert the dataframe
            value.to_excel(writer, sheet_name=sheet_name, startcol=3, startrow=0) 


# **Loan Characteristics (HMDA)**

## Categorical

In [ ]:
# derived_loan_product_type
hmda_dict_lar_df[['derived_loan_product_type']].value_counts(normalize = True, dropna = False)

In [ ]:
# loan_type
hmda_dict_lar_df[['loan_type']].value_counts(normalize = True, dropna = False)

In [ ]:
# loan_purpose
hmda_dict_lar_df[['loan_purpose']].value_counts(normalize = True, dropna = False)

In [ ]:
# lien_status
hmda_dict_lar_df[['lien_status']].value_counts(normalize = True, dropna = False)

In [ ]:
# open_end_line_of_credit
hmda_dict_lar_df[['open_end_line_of_credit']].value_counts(normalize = True, dropna = False)

In [ ]:
# business_or_commercial_purpose
hmda_dict_lar_df[['business_or_commercial_purpose']].value_counts(normalize = True, dropna = False)

## Numeric

In [ ]:
# loan_amount
hmda_dict_lar_df['loan_amount'].describe()

In [ ]:
# loan_amount 
hmda_dict_lar_df['loan_amount'].isnull().sum()

In [ ]:
# interest_rate na value count
hmda_dict_lar_df['interest_rate'].replace('Exempt', np.nan).apply(float).describe()

In [ ]:
# interest_rate na value count
hmda_dict_lar_df[['interest_rate']].replace('Exempt', np.nan).isnull().sum()

In [ ]:
# origination_charges
hmda_dict_lar_df['origination_charges'].replace('Exempt', np.nan).apply(float).describe()

In [ ]:
# origination_charges na value count
hmda_dict_lar_df[['origination_charges']].replace('Exempt', np.nan).isnull().sum()

# **Borrower characteristics (HMDA)** 

## Categorical

In [ ]:
# debt_to_income_ratio
hmda_dict_lar_df['debt_to_income_ratio'].value_counts(normalize = True, dropna = False)

In [ ]:
# applicant_credit_score_type
hmda_dict_lar_df['applicant_credit_score_type'].value_counts(normalize = True, dropna = False)

In [ ]:
# applicant_sex
hmda_dict_lar_df['applicant_sex'].value_counts(normalize = True, dropna = False)

In [ ]:
# applicant_age
hmda_dict_lar_df['applicant_age'].value_counts(normalize = True, dropna = False)

# **Loan characteristics (SBA)**

## Categorical

In [ ]:
sba_data['DeliveryMethod'].value_counts(normalize = True, dropna = False)  

In [ ]:
sba_data['NaicsDescription'].value_counts(normalize = True, dropna = False)

In [ ]:
sba_data['CongressionalDistrict'].value_counts(normalize = True, dropna = False)

In [ ]:
sba_data['BusinessType'].value_counts(normalize = True, dropna = False)

In [ ]:
sba_data['BusinessAge'].value_counts(normalize = True, dropna = False)

## Numeric

In [ ]:
sba_data['Total loan amount'].describe()

In [ ]:
sba_data['SBAGuaranteedApproval'].describe()

In [ ]:
sba_data['InitialInterestRate'].describe()

In [ ]:
sba_data['TermInMonths'].describe()

In [ ]:
sba_data['JobsSupported'].describe()

# **FFIEC Analysis**

In [ ]:
ffiec_data.groupby(['CRA poverty criteria','Tract Income Indicator'])\
['Tract median family income as a percentage of the MSA/MD median family income. 2 decimal places, truncated.']\
.describe().reset_index()[['CRA poverty criteria',\
                           'Tract Income Indicator',\
                           'min','max']].sort_values(by = ['CRA poverty criteria',\
                           'Tract Income Indicator'], ascending = False)